# Complete LLM Handbook

Install Dependencies to run code blocks

In [ ]:
%pip install tensorflow keras keras_nlp matplotlib numpy pandas scikit-learn

## Sections 

### [1 Tokenization, vocab, sequence formatting](#1-Tokenization,-vocab,-sequence-formatting)
- [1.1 Byte level words vs Subword tokenization](#1.1-Byte-level-words-vs-Subword-tokenization)
  - [1.1.1 Byte level tokenization](#1.1.1-Byte-level-tokenization)
  - [1.1.2 Subword tokenization (BPE, WordPiece, SentencePiece)](#1.1.2-Subword-tokenization-(BPE,-WordPiece,-SentencePiece))
- [1.2 Token Ids and Vocabulary Size](#1.2-Token-Ids-and-Vocabulary-Size)
- [1.3 Padding and Masking](#1.3-Padding-and-Masking)
- [1.4 Special Tokens](#1.4-Special-Tokens)
- [1.5 Sequence Packing and Contiguous Streams](#1.5-Sequence-Packing-and-Contiguous-Streams)
- [1.6 Sliding Window Chunking](#1.6-Sliding-Window-Chunking)
- [1.7 Complete Example: Combining All Tokenization Steps](#1.7-Complete-Example:-Combining-All-Tokenization-Steps)

### [2 Embedding and Unembedding](#2-Embedding-and-Unembedding)
- [2.1 Word embedding lookup tables](#2.1-Word-embedding-lookup-tables)
- [2.2 Unembedding and tied embeddings](#2.2-Unembedding-and-tied-embeddings)
  - [2.2.1 Unembedding](#2.2.1-Unembedding)
  - [2.2.2 Tied Embeddings](#2.2.2-Tied-Embeddings)
- [2.3 Why positional representations are required](#2.3-Why-positional-representations-are-required)
- [2.4 Positional encoding types](#2.4-Positional-encoding-types)
  - [2.4.1 Sinusoidal positional encoding](#2.4.1-Sinusodal-positional-encoding)
  - [2.4.2 Learned positional embeddings](#2.4.2-Learned-positional-embeddings)
  - [2.4.3 Rotary Position Embeddings (RoPE)](#2.4.3-Rotary-Position-Embeddings-(RoPE))
  - [2.4.4 ALiBi](#2.4.4-ALiBi)
- [2.5 How positional encoding interacts with attention](#2.5-How-positional-encoding-interacts-with-attention)
- [2.6 Embedding scaling by sqrt(d_model)](#2.6-Embedding-scaling-by-sqrt(d_model))
- [2.7 Complete Example: Combining All Embedding Steps](#2.7-Complete-Example:-Combining-All-Embedding-Steps)

### [3 Attention](#3-Attention)
- [3.1 Query, Key, Value fundamentals](#3.1-Query,-Key,-Value-fundamentals)
- [3.2 Dot product attention](#3.2-Dot-product-attention)
- [3.3 Why divide by √dk](#3.3-Why-divide-by-√dk)
- [3.4 Causal masking and autoregressive behavior](#3.4-Causal-masking-and-autoregressive-behavior)
- [3.5 Softmax details and numerical stability](#3.5-Softmax-details-and-numerical-stability)
- [3.6 Multi-head attention](#3.6-Multi-head-attention)
- [3.7 Attention complexity](#3.7-Attention-complexity)
- [3.8 Memory layout and tensor shapes](#3.8-Memory-layout-and-tensor-shapes)
- [3.9 Flash Attention (conceptual)](#3.9-Flash-Attention-(conceptual))
- [3.10 Self attention vs cross attention](#3.10-Self-attention-vs-cross-attention)
- [3.11 Key-value caching for generation](#3.11-Key-value-caching-for-generation)
- [3.12 TensorFlow Examples](#3.12-Tensorflow-Example-(no-raw-kernal-high-level-usage))
  - [3.12.1 Single-head attention](#3.12.1-Single-head-attention-(conceptual-TF))
  - [3.12.2 Multi-head attention](#3.12.2-Multi-head-attention-(high-level))
  - [3.12.3 Causal mask](#3.12.3-Causal-mask)
  - [3.12.4 Multi-head attention demo](#3.12.4-Multi-Head-Attention-in-Action:-Example)
  - [3.12.5 What transformers do: context integration](#3.12.5-What-Transformers-Do-and-what-do-(context-integration))

### [4 Feed Forward Networks (MLP block)](#4-Feed-Forward-Networks-(MLP-block))
- [4.1 What the FFN is really doing](#4.1-What-the-FFN-is-really-doing)
- [4.2 Two-layer MLP structure](#4.2-Two-layer-MLP-structure)
- [4.3 Activation functions and why GELU matters](#4.3-Activation-functions-and-why-GELU-matters)
  - [4.3.1 ReLU](#4.3.1-ReLU)
  - [4.3.2 GELU (Gaussian Error Linear Unit)](#4.3.2-GELU-(Gaussian-Error-Linear-Unit))
- [4.4 Modern variants: SwiGLU](#4.4-Modern-variants:-SwiGLU)
- [4.5 Dropout and residual scaling](#4.5-Dropout-and-residual-scaling)
- [4.6 TensorFlow: Configurable FFN block (high level)](#4.6-TensorFlow:-Configurable-FFN-block-(high-level))

### [5 Normalization and Stabilization in Transformers](#5.-Normalization-and-Stabilization-in-Transformers)
- [5.1 Why normalization is critical in deep transformers](#5.1-Why-normalization-is-critical-in-deep-transformers)
- [5.2 LayerNorm inside transformers](#5.2-LayerNorm-inside-transformers)
- [5.3 Post-Norm vs Pre-Norm](#5.3-Post-Norm-vs-Pre-Norm)
  - [5.3.1 Post-Norm (original Transformer)](#5.3.1-Post-Norm-(original-Transformer))
  - [5.3.2 Pre-Norm (modern standard)](#5.3.2-Pre-Norm-(modern-standard))
- [5.4 RMSNorm](#5.4-RMSNorm)
- [5.5 Why normalization stabilizes attention and FFNs](#5.5-Why-normalization-stabilizes-attention-and-FFNs)
- [5.6 Initialization strategies for stability](#5.6-Initialization-strategies-for-stability)
- [5.7 Residual connection depth scaling](#5.7-Residual-connection-depth-scaling)
- [5.8 TensorFlow: Pre-Norm transformer components (high level)](#5.8-TensorFlow:-Pre-Norm-transformer-components-(high-level))

### [6 The Transformer Decoder Block](#6-The-Transformer-Decoder-Block)
- [6.1 Decoder block order and why it is this way](#6.1-Decoder-block-order-and-why-it-is-this-way)
- [6.2 Shape propagation through the block](#6.2-Shape-propagation-through-the-block)
- [6.3 Dropout behavior inside the decoder](#6.3-Dropout-behavior-inside-the-decoder)
- [6.4 Trainable parameter count intuition](#6.4-Trainable-parameter-count-intuition)
- [6.5 How to tune hidden sizes](#6.5-How-to-tune-hidden-sizes)
- [6.6 Why decoder-only models work for LLMs](#6.6-Why-decoder-only-models-work-for-LLMs)
- [6.7 TensorFlow: Full DecoderBlock implementation](#6.7-TensorFlow:-Full-DecoderBlock-implementation)

### [7 Putting blocks together into a Transformer Decoder](#7-Putting-blocks-together-into-a-Transformer-Decoder)
- [7.1 Stacking N decoder blocks](#7.1-Stacking-N-decoder-blocks)
- [7.2 Why weights are not shared across layers](#7.2-Why-weights-are-not-shared-across-layers)
- [7.3 How input embeddings flow into the decoder](#7.3-How-input-embeddings-flow-into-the-decoder)
- [7.4 How logits are produced](#7.4-How-logits-are-produced)
- [7.5 TensorFlow: Tiny GPT style model from scratch](#7.5-TensorFlow:-Tiny-GPT-style-model-from-scratch)

### [8 Training LLMs](#8-Training-LLMs)
- [8.1 Next-token prediction objective](#8.1-Next-token-prediction-objective)
- [8.2 Teacher forcing](#8.2-Teacher-forcing)
- [8.3 Loss calculation and padding masks](#8.3-Loss-calculation-and-padding-masks)
- [8.4 Gradient accumulation](#8.4-Gradient-accumulation)
- [8.5 Gradient clipping](#8.5-Gradient-clipping)
- [8.6 Learning rate schedules](#8.6-Learning-rate-schedules)
- [8.7 Mixed precision training](#8.7-Mixed-precision-training)
  - [8.7.1 Loss Scaling](#8.7.1-Loss-Scaling)
- [8.8 TensorFlow: Custom autoregressive training loop](#8.8-TensorFlow:-Custom-autoregressive-training-loop)
  - [8.8.1 Enable mixed precision](#8.8.1-Enable-mixed-precision)
  - [8.8.2 Loss function with padding mask](#8.8.2-Loss-function-with-padding-mask)
  - [8.8.3 Custom training step](#8.8.3-Custom-training-step)
  - [8.8.4 Optimizer setup](#8.8.4-Optimizer-setup)
  - [8.8.5 Load built-in dataset and create a small model](#8.8.5-Load-built-in-dataset-and-create-a-small-model)
  - [8.8.6 Training loop](#8.8.6-Training-loop)

### [9 Generation Controls for LLMs](#9-Generation-Controls-for-LLMs)
- [9.1 Greedy decoding](#9.1-Greedy-decoding)
- [9.2 Sampling](#9.2-Sampling)
- [9.3 Temperature](#9.3-Temperature-(ml-notes-p-53))
- [9.4 Top-k sampling](#9.4-Top-k-sampling)
- [9.5 Top-p (nucleus) sampling](#9.5-Top-p-(nucleus)-sampling)
- [9.6 Repetition penalty](#9.6-Repetition-penalty)
- [9.7 Length penalty](#9.7-Length-penalty)
- [9.8 Stopping conditions](#9.8-Stopping-conditions)
- [9.9 Key-value cache for speed](#9.9-Key-value-cache-for-speed)
- [9.10 Generation in tf.function](#9.10-Generation-in-tf.function)
- [9.11 TensorFlow: Full autoregressive generation function](#9.11-TensorFlow:-Full-autoregressive-generation-function)

### [10 Scaling and regularization](#10-Scaling-and-regularization)
- [10.1 Parameter scaling laws](#10.1-Parameter-scaling-laws)
- [10.2 Dropout placement](#10.2-Dropout-placement)
- [10.3 Weight decay](#10.3-Weight-decay)
- [10.4 Learning rate schedules](#10.4-Learning-rate-schedules)
- [10.5 Warmup linear schedules](#10.5-Warmup-linear-schedules)
- [10.6 Cosine decay](#10.6-Cosine-decay)
- [10.7 Adam vs AdamW](#10.7-Adam-vs-AdamW)
- [10.8 Tokenization impact on scale](#10.8-Tokenization-impact-on-scale)
- [10.9 Transformer depth and expressivity](#10.9-Transformer-depth-and-expressivity)
- [10.10 Putting it together: real vs toy training](#10.10-Putting-it-together:-real-vs-toy-training)

### [11 Evaluation and Metrics for LLMs](#11-Evaluation-and-Metrics-for-LLMs)
- [11.1 Perplexity](#11.1-Perplexity)
- [11.2 Bits per character (BPC)](#11.2-Bits-per-character-(BPC))
- [11.3 Loss curves](#11.3-Loss-curves)
- [11.4 Out-of-distribution evaluation](#11.4-Out-of-distribution-evaluation)
- [11.5 Token-level accuracy](#11.5-Token-level-accuracy)
- [11.6 Sampling-based evaluation](#11.6-Sampling-based-evaluation)
- [11.7 Practical evaluation workflow](#11.7-Practical-evaluation-workflow)
- [11.8 When metrics lie](#11.8-When-metrics-lie)
- [11.9 End-to-end evaluation of a pretrained GPT-2 (TensorFlow)](#11.9-End-to-end-evaluation-of-a-pretrained-GPT-2-(TensorFlow))


## 1 Tokenization, vocab, sequence formatting

### 1.1 Byte level words vs Subword tokenization 

**Why**: Transformers cannot process raw text, text must be converted into numbers. The way we break text into tokens affects efficency, generalization and memory usage

#### 1.1.1 Byte level tokenization

- works at the byte level (0-255)
- Real world usage: GPT-2 uses byte pair encoding (BPE) at byte level
- Pros:
  - Handels any charecter, any language, emojis, symbol
  - no OOV (out of vocab) tokens
- Cons:
  - Toekn sequences can be longer -> means more compute 
- Example: "hello 👋" –> [104, 101, 108, 108, 111, 32, 240, 159, 145, 139] (in token ids where range is 0 -> vocab_size)

#### 1.1.2 Subword tokenization (BPE, WordPeice, SentencePiece)

-  Breaks Text into frequent subwords insted of characters or words. 
-  Example: 
   -  "unhappiness" -> ["un", "happi", "ness"] -> [217, 9812, 403] # in token ids (range is 0 -> vocab size)
- Pros: 
  - Shorter sequences than byte
  - Can handle rare words via subword decomposition (breaking unknown words into known smaller parts)
- Cons:
  - some complexity in building vocab and handling edge cases
  
**NOTE:** LLM's often use subword BPE (BPE applied at the subword level) it iteratively merges the most frequent character or subword pairs to build a vocabulary, balancing between character-level and word-level tokenization

### 1.2 Token Ids and Vocabulary Size
- After Tokenization, each token is mapped to a integer ID using a vocabulary
- Vocabulary size (V) is very important
  - Larger V -> model must have a bigger embedding matrix (page 50 in written notes) -> more parameters (hence a larger model)
  - Smaller V -> more subword splitting (words broken into more pieces) -> longer sequences -> slower training (but smaller model size)
- Typical LLM vocab sizes: 30K-100K for english models 
- Example: In TensorFlow, keras_nlp.tokenizers handles both mapping tokens → IDs and IDs → tokens.

```py
from keras_nlp.tokenizers import BytePairTokenizer

tokenizer = BytePairTokenizer(vocabulary=["hello", "world", "un", "happi", "ness", "<PAD>", "<BOS>", "<EOS>"])
tokens = tokenizer.tokenize(["hello world", "unhappiness"])
token_ids = tokenizer(tokens)
print(token_ids)
```

**How Keras NLP Tokenizers Handle Token ↔ ID Mapping** Under the hood, Keras NLP tokenizers maintain two key data structures (`token_to_id` and `id_to_token`) for bidirectional mapping. When you call `tokenizer.tokenize(text)`, it returns tokens as strings; `tokenizer(text)` returns token IDs; and `tokenizer.detokenize(ids)` converts IDs back to text. The vocabulary is built during training or loaded from a pre-trained model, with special tokens (PAD, UNK, BOS, EOS) typically assigned fixed IDs at the beginning.


### 1.3 Padding and Masking 
1. Padding: Short sequences are extended with PAD tokens to match the longest sequence in a batch, enabling efficient parallel processing (e.g., `[5, 10, 15]` → `[5, 10, 15, <PAD>, <PAD>]`)

2. Attention Masking: Tells the transformer which positions to ignore during attention.
- **No Mask (Bidirectional)**: All tokens attend to all tokens; used in BERT for full context understanding
- **Causal Mask (Autoregressive)**: Each token only attends to previous tokens; used in GPT to prevent future information leakage during training
- **Padding Mask**: Masks PAD tokens so they don't affect attention scores; combined with other masks in most models


In [ ]:
import tensorflow as tf

# Example: batch of token IDs (here each array of token ids in a batch is a sqeuence i.e one example, by spliting in batches we can proccess in parallel)
batch = tf.ragged.constant([
    [1, 2, 3],
    [4, 5]
])
padded = batch.to_tensor(default_value=0) # Output: [[1, 2, 3], [4, 5, 0]]  <- 0 is the PAD token ID (these are the new tokens)
mask = tf.cast(padded != 0, tf.int32) # Output: [[1, 1, 1], [1, 1, 0]]  <- tells attention to ignore the last position in sequence 2 (this is the attention scores not token values)

### 1.4 Special Tokens (see ML notes page 121)
- `<BOS>`: Beginning of sequence (marks where a sequence starts)
- `<EOS>`: End of sequence (marks where a sequence ends)
- `<PAD>`: Padding (fills shorter sequences to match batch length)
- `<UNK>`: Unknown/ out of vocab token (represents words not in vocabulary)
- etc

**usage in training**
``` text
Input:  <BOS> hello world <EOS> <PAD> <PAD>    # BOS is fed as a conditioning token ((a special input token that provides initial context/prompt for the model; the model conditions its next-token predictions on it but is not trained to predict it)) EOS is included so the model learns to predict sequence end PADs fill to uniform length

Target: hello world <EOS> <PAD> <PAD> <PAD>   # Target = input shifted left (model predicts the next token at each step, including EOS); PADs fill to uniform length

Mask:   1 1 1 1 0 0 0                          # Mask=1 for positions to compute loss (we compute loss for real tokens and EOS), 0 for PADs
```

### 1.5 Sequence Packing and Contiguous Streams 
- Why: LLM training is compute-heavy, to use memory efficiently, multiple short examples can be concatenated into a single long sequence and then chunked
- Benefits: 
  - Reduces wasted padding
  - keepinh sequences dense for attention
  
**Example (pseudo)**
```text
Examples: ["hello", "world"], ["goodbye", "moon"]
Packed sequence: "hello world goodbye moon"
```
- Then split into fixed length chunks (ex: 8 tokens per chunk) for processing

### 1.6 Sliding Window Chunking 
- When text is too long to fit in memory, we create overlapping windown to preserve context
- why: prevents cutting off dependencies between sequences.
- Example Sequence length = 6, chunk size = 4, stride = 2
``` text
Sequence: [A B C D E F] (len = 6)

Chunk 1: Start at position 0 → [A B C D] (len = 4 beacuse chunk size = 4)
Chunk 2: Start at position 0 (position) + 2(stride) = 2 → [C D E F] (move the window by 2, keeps last 2 tokens of last chink in new chunk)

Chunks:  [A B C D], [C D E F]

Result Sequence: [A B C D], [C D E F]
                      ↑overlap↑
```
- Edge case: If the final window doesn't have enough tokens (e.g., only 3 tokens left for chunk_size=4), you either pad it with `<PAD>` tokens or discard it depending on your training strategy.
- overlapping ensures context continuity for training 
- common in LLM pretraining 
  

### 1.7 Complete Example: Combining All Tokenization Steps

This example demonstrates the entire pipeline from raw text to training-ready sequences, incorporating all concepts from 1.1-1.6.

In [ ]:
import tensorflow as tf
import keras_nlp

# Load pretrained tokenizer (handles vocab, special tokens, BPE subword)
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")

# Create dataset pipeline: tokenization → chunking → padding
dataset = (
    tf.data.Dataset.from_tensor_slices(["Hello world", "Goodbye moon"])
    .map(tokenizer)  # Tokenize: text → IDs
    .unbatch()  # Flatten to token stream (packing)
    .batch(8, drop_remainder=False)  # Chunk into sequences of 8 tokens
    .map(lambda x: (x[:-1], x[1:]))  # Create (input, target) pairs
    .padded_batch(2, padded_shapes=([None], [None]))  # Pad and batch
)

# Usage:
inputs, targets = next(iter(dataset))
print(tokenizer.detokenize(inputs[0]))
print(tokenizer.detokenize(targets[0]))

**OUTPUT:**
```
Hello worldGoodbye
worldGoodbye moon
```

**EXPLANATION:**

The dataset variable is a `tf.data.Dataset` pipeline that transforms raw text into training-ready (input, target) pairs. It's a lazy iterator (doesn't process until called).

**Pipeline steps:**
1. **from_tensor_slices**: Creates dataset from list of strings
2. **map(tokenizer)**: Converts each text → token IDs (e.g., "Hello" → [15496, 995])
3. **unbatch()**: Flattens all sequences into one continuous token stream (sequence packing)
4. **batch(8)**: Groups tokens into chunks of 8 (creates fixed-length sequences)
5. **map(lambda)**: Splits each chunk into (input, target) where target = input shifted left
6. **padded_batch(2)**: Groups 2 sequences into a batch, pads shorter ones to match length

**How next-token prediction works:**

The model predicts the NEXT token at EACH position, not just the last one:
- Position 0: Given "Hello" → predict "world"
- Position 1: Given "Hello world" → predict "Goodbye"  
- Position 2: Given "Hello worldGoodbye" → predict "moon"

So the target sequence shows what should be predicted at each step. The entire target = input shifted left by 1 token (each target is the next token).

**Chunking Strategy Comparison:**

Token stream: `[A, B, C, D, E, F, G, H, I, J]`

**Fixed batch** (current): `.batch(4)`
- Chunk 1: `[A, B, C, D]` (tokens 0-3)
- Chunk 2: `[E, F, G, H]` (tokens 4-7)
- Chunk 3: `[I, J]` (tokens 8-9)
- → No overlap, each token appears once

**Sliding window**: `.window(size=4, shift=2, drop_remainder=True)`
- Chunk 1: `[A, B, C, D]` (tokens 0-3)
- Chunk 2: `[C, D, E, F]` (tokens 2-5, overlaps last 2 from chunk 1)
- Chunk 3: `[E, F, G, H]` (tokens 4-7, overlaps last 2 from chunk 2)
- → Overlap preserves context across chunks, useful for long documents

**IMPORTANT: Both strategies train on next-token prediction at EVERY position!**

**Fixed batch:**
- Chunk 1: `[A,B,C,D]` → trains: (A→B), (A,B→C), (A,B,C→D)
- Chunk 2: `[E,F,G,H]` → trains: (E→F), (E,F→G), (E,F,G→H)
- Each token appears ONCE

**Sliding window:**
- Chunk 1: `[A,B,C,D]` → trains: (A→B), (A,B→C), (A,B,C→D)
- Chunk 2: `[C,D,E,F]` → trains: (C→D), (C,D→E), (C,D,E→F)
- Tokens C and D appear TWICE (extra training for better context)

**Key Takeaway:** The chunking method only affects which tokens are grouped together, not how training works. Sliding window gives overlapping tokens extra exposure for better long-range dependencies.

## 2 Embedding and Unembedding

### 2.1 Word embedding lookup tables

**What a Embedding is**

- An embedding is just a trainable lookup table:
  - Shape: (vocab_size, d_model)
  - Input: token ID's of shape (batch, seq_len)
  - Output: vectors of shape (batch, seq_len, d_model)

Each token ID indexes one row, the learning happens because gradients update those rows based on prediction error

**Traning Proccess**: How Embeddings Are Learned

1. **Random Initialization**: Embedding table starts with random values (shape: `vocab_size × d_model`)

2. **Forward Pass**: 
    - Token IDs → Look up embeddings → Feed through transformer → Predict next token

3. **Loss Calculation**:
    - Compare prediction to actual next token
    - Compute cross-entropy loss

4. **Backpropagation**:
    - Gradients flow back through the entire model
    - Embedding table rows get gradient updates based on which tokens were used

5. **Update Rule** (simplified):
    ```
    embedding[token_id] -= learning_rate × gradient[token_id]
    ```

**Key Insight**: Tokens that appear in similar contexts will develop similar embeddings because they receive similar gradient updates. For example:
- "cat" and "dog" → often surrounded by words like "pet", "animal" → embeddings become similar
- "king" and "queen" → share contexts like "royal", "throne" → learn related representations

The model learns embeddings **jointly** with all other parameters (attention weights, feedforward layers) to minimize prediction error across the entire training corpus.

**Example Matrix:**
``` text
vocab_size = 50000  # Total unique tokens in vocabulary
d_model = 512       # Each token → 512-dimensional vector
embedding_table = tf.Variable(shape=(50000, 512))
# If token ID = 42, its embedding is embedding_table[42] (a 512-length vector)
```

In short the embedding is the models repersentation of a token, attention uses these repersentations to operate

**TensorFlow Example**
``` py
import tensorflow as tf

class TokenEmbedding(tf.keras.layers.Layer): # inherit tf Layer Class
    def __init__(self, vocab_size, d_model): # init model
        super().__init__()
        self.embedding = tf.keras.layers.Embedding( # create embedding layer
            input_dim=vocab_size,
            output_dim=d_model
        )

    # override call function with out custom embeddign layer
    def call(self, token_ids):
        return self.embedding(token_ids)
```

Example IO:
```text
input:  [12,   431,   98] # token ID's

# each token gets a embedding returned (output[i] = E[token_ids[i]]) where 'E' is teh embedding matrix
output: [
  E[12],     # ∈ R^d_model here 'R' is (real numbers) and 'd_model' is the number of dimentions in the model its a hyperparameter you chose (like 64, 128, 512, etc) for ex R^4 = [x1, x2, x3, x4] x can be any real number
  E[431],    # ∈ R^d_model
  E[98]      # ∈ R^d_model
]


```




### 2.2 Unembedding and tied embeddings

#### 2.2.1 Unembedding

Unembedding converts each transformer output vector into a score for every token in the vocabulary so the model can predict which token comes next. we need this at the output of the transformer
```
Vocabulary: ["hello", "world", "cat", "dog", ...] (ex 50000 words)
                  ↓       ↓      ↓     ↓
Logits:          [2.1,   0.5,  -1.2,  3.4, ...]  ← Higher score = more likely next token (50000 scores one for each token)
```
this is done with linear projection.

- Input: (batch, seq_len, d_model)
- Weight: (d_model, vocab_size)
- Output: (batch, seq_len, vocab_size)

**NOTE:** this layer is also called the unembedding or LM head

**NOTE:** The converstion of the token ID back to a final word is the tokenizers job do not mistake unebedding for that


**Training Process**: How Unembedding Weights Are Learned

1. **Random Initialization**: Unembedding matrix starts with random values (shape: `d_model × vocab_size`)

2. **Forward Pass**:
    - Transformer outputs → (batch, seq_len, d_model)
    - Matrix multiply with unembedding weights → (batch, seq_len, vocab_size)
    - Apply softmax → probability distribution over vocabulary

3. **Loss Calculation**:
    - Compare predicted probabilities to actual next token (one-hot encoded)
    - Compute cross-entropy loss: `loss = -log(P(correct_token))`

4. **Backpropagation**:
    - Gradients flow back from loss through softmax and unembedding layer
    - Each row of the unembedding matrix (corresponding to one output dimension) gets updated based on prediction errors

5. **Update Rule** (simplified):
    ```
    unembedding_weights -= learning_rate × gradient
    ```

**Key Insight**: The unembedding layer learns which transformer output patterns correspond to which tokens. If the model frequently outputs vectors in a certain direction when "cat" should be next, those weights get strengthened to produce higher logits for "cat".

**Example:**
``` text
d_model = 512
vocab_size = 50000
unembedding_matrix = tf.Variable(shape=(512, 50000))

# Transformer output: (batch=1, seq_len=1, d_model=512)
# After matmul: (batch=1, seq_len=1, vocab_size=50000)
# Each position gets 50000 scores (one per possible next token)

# Mathamatically

# Hidden vector h ∈ R^d_model # from tansfromer 
# Unembedding matrix W ∈ R^(d_model × vocab_size) # learned matrix
# Logits l = h · W   → l ∈ R^vocab_size # 50000 logits as a result
```

**The flow so far is as follows**: Token ID → Embedding → Transformer → Hidden Vector → Unembedding → Logits → Token ID → Word (where we pick the highest logit as next word)

**Tensorflow Example**

```py
import tensorflow as tf

class TokenUnembedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        # Linear layer without bias: projects hidden vectors to vocab logits
        self.dense = tf.keras.layers.Dense(vocab_size, use_bias=False)

    def call(self, hidden_states):
        # hidden_states: (batch, seq_len, d_model)
        # logits: (batch, seq_len, vocab_size)
        logits = self.dense(hidden_states)
        return logits

```

#### 2.2.2 Tied Embeddings

Modern LLMs tie the input embedding matrix and output projection weights 

- Why it works:
  - the same geometric space is used for reading and writing tokens 
  - reduces parameters 
  - improves sample efficiency and stability
- Mathamatically
  - input embedding: E[token_id]
  - output logits: h · Eᵀ (h is the hidden vector)
  
This enforces symmetry between encoding and decoding

**Tensorflow example**
``` py
class TiedOutputProjection(tf.keras.layers.Layer):
    def __init__(self, embedding_layer):
        super().__init__()
        self.embedding_layer = embedding_layer

    def call(self, hidden_states):
        embedding_matrix = self.embedding_layer.embedding.embeddings
        logits = tf.einsum("btd,vd->btv", hidden_states, embedding_matrix)
        return logits
```

### 2.3 Why positional representations are required (ML notes pg 112)

**Self attention is permutation-invariant.**

That means:
- “cat sat mat” and “mat sat cat” look identical without position.
- Order must be injected explicitly.

Positions noting sequence index are added or applied to embeddings before attention.

**Key idea:**
Token meaning + position meaning = input representation


### 2.4 Positional encoding types

#### 2.4.1 Sinusodal positional encoding (ML Notes pg 112)

**Concept**
- Deterministic, non-trainable
- Uses sine and cosine at different frequencies
- Allows extrapolation to longer sequences

**Formulas:**
- PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
- PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))

**Why it works:**
- Relative positions can be inferred from linear combinations
- Attention can compute distance relationships

**Tensorflow Example:**
```py
import math

def sinusoidal_position_encoding(seq_len, d_model):
    position = tf.range(seq_len, dtype=tf.float32)[:, None]
    div_term = tf.exp(
        tf.range(0, d_model, 2, dtype=tf.float32) *
        -(math.log(10000.0) / d_model)
    )

    pe = tf.zeros((seq_len, d_model))
    pe = tf.tensor_scatter_nd_update(
        pe,
        indices=tf.range(0, d_model, 2)[:, None],
        updates=tf.sin(position * div_term)
    )
    pe = tf.tensor_scatter_nd_update(
        pe,
        indices=tf.range(1, d_model, 2)[:, None],
        updates=tf.cos(position * div_term)
    )
    return pe
```

**added directly to token embeddings**

#### 2.4.2 Learned positional embeddings (ML notes pg 112)
**Concept**
- Position IDs get their own embedding table
- Fully learned
- Used in GPT-2, BERT

**Pros:**
- Flexible
- Often better on fixed context lengths

**Cons:**
- Cannot extrapolate beyond max length trained

**Tensorflow Example:**
```py
class LearnedPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(max_len, d_model)

    def call(self, seq_len):
        positions = tf.range(seq_len)
        return self.embedding(positions)
```

#### 2.4.3 Rotary Position Embeddings (RoPE)
This is where modern LLMs diverge from early transformers.

**Core idea:** RoPE rotates query and key vectors in embedding space based on position.

**Key properties:**
- Position information is applied inside attention
- Enables relative position reasoning
- Scales well to long contexts

Instead of adding position vectors, we rotate pairs of dimensions:
```text
embedding vector = [x1, x2] → rotation by angle θ( determined by tokens position)
x1' = x1*cosθ - x2*sinθ
x2' = x1*sinθ + x2*cosθ
x_rotated = [x1', x2']
```

**Why this is powerful**
- Dot products encode relative distance naturally when dot product of K,Q are rotated
- No learned position embeddings
- Better extrapolation

**TensorFlow implementation**
```py
def rotary_embedding(x, seq_len):
    d_model = x.shape[-1]
    half = d_model // 2

    freqs = tf.exp(
        -tf.range(0, half, dtype=tf.float32) / half * tf.math.log(10000.0)
    )
    positions = tf.range(seq_len, dtype=tf.float32)
    angles = positions[:, None] * freqs[None, :]

    sin = tf.sin(angles)
    cos = tf.cos(angles)

    x1 = x[..., :half]
    x2 = x[..., half:]

    rotated = tf.concat(
        [x1 * cos - x2 * sin,
         x1 * sin + x2 * cos],
        axis=-1
    )
    return rotated
```
**Applied to queries and keys only, never values.**

#### 2.4.4 ALiBi
**Concept**
- No position embeddings at all
- Adds a linear bias to attention scores
- Penalizes distant tokens

**Attention score becomes:**
``` text
QKᵀ / sqrt(d) + bias(distance)
```

**Why it matters:**
- Extremely simple
- Strong extrapolation to long sequences
- Used in MPT and others

**NOTE:** ALiBi changes attention, not embeddings.

**You do not add position vectors at input.**




### 2.5 How positional encoding interacts with attention
**Important clarity:**
- Additive encodings modify token representations before attention.
- RoPE modifies query and key geometry.
- ALiBi modifies attention logits directly.

**All three inject order, but at different stages.**

**This choice affects:**
- Long context scaling
- Memory behavior
- Generalization beyond training length

### 2.6 Embedding scaling by sqrt(d_model)

When embeddings are initialized, their variance is small.

If we add positional encodings directly, they can dominate early training.

**Standard fix:**
```text
x = embedding(token_ids) * sqrt(d_model)
x = x + position_encoding
```

**This ensures:**
- Token identity dominates initially
- Position is a refinement, not the signal

**TF snippet**
```py
x = token_embedding(token_ids)
x *= tf.math.sqrt(tf.cast(d_model, tf.float32))
x += position_encoding
```

**This is not cosmetic. It stabilizes training.**


### 2.7 Complete Example: Combining All Embedding Steps

This example demonstrates the entire embedding pipeline from token IDs to final logits, a code example for 2.1-2.6.

In [ ]:
import tensorflow as tf
import keras_nlp

# Load pretrained tokenizer and model
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
backbone = keras_nlp.models.GPT2Backbone.from_preset("gpt2_base_en")

# Example text: Notice "cat" appears at different positions
text = "The cat sat on the mat. I love my cat very much."

# Step 1: Tokenize (from section 1)
token_ids = tokenizer(text) # Token IDs: [ 464 3797 3332  319  262 2603   13  314 1842  616 3797  845  881   13]

# Step 2: Token Embedding (2.1) - Convert IDs to vectors
token_embedding = backbone.token_embedding # Token Embeddings shape: (14, 768), Each token → 768-dimensional vector
token_embeds = token_embedding(token_ids)

# Step 3: Position Embedding (2.4) - Add position information
position_embedding = backbone.position_embedding # Position Embeddings shape: (14, 768)
pos_embeds = position_embedding(token_embeds)

# Step 4: Combine (2.5) - Token meaning + Position
final_embeds = token_embeds + pos_embeds #  Final Embeddings (token + position): (14, 768)

# ============================================================
# Step 5: Unembedding (2.2) - Project back to vocabulary
# ============================================================
print(f"\n5. Unembedding (tied weights):")
# NOTE: We're using GPT-2's PRETRAINED embedding weights
# These were learned on billions of tokens, so predictions are meaningful
# In practice, you'd train these from scratch on your data
embedding_matrix = token_embedding.embeddings  # Reuse same pretrained weights
logits = tf.matmul(final_embeds, embedding_matrix, transpose_b=True) # Logits shape: (14, 50257)
predicted_token_ids = tf.argmax(logits, axis=-1)  # Predicted Token IDs shape: (14,)

**SUMMARY: What we covered from sections 2.1-2.6**

**1. TOKEN EMBEDDING (2.1):**
- Input: Token IDs [464, 3797, ...]
- Output: Dense vectors [(768 dims), (768 dims), ...]
- → Each token gets a learned representation

**2. POSITION EMBEDDING (2.4):**
- Learned embeddings that encode position
- Position 0 ≠ Position 10 (different vectors)
- → Tells model WHERE each token is

**3. COMBINING (2.5):**
- final = token_embed + pos_embed
- → Same word at different positions has different final representations

**4. UNEMBEDDING (2.2):**
- Projects embeddings → vocabulary logits
- Uses TIED WEIGHTS (same matrix as input embeddings)
- → Predicts next token distribution

**IMPORTANT:** This example uses GPT-2's PRETRAINED embeddings!
- These weights were learned on billions of tokens of web text
- That's why predictions are meaningful (not random)
- In your own model, you'd train these from scratch on your data
- Training process: same backpropagation as described in 2.1

**Key Insight:** Without position embeddings, "cat sat mat" and "mat sat cat" would look identical to the model. Position info is CRITICAL.

**Note:** We haven't covered transformers yet - that processes these embeddings! The transformer would sit between step 4 (final embeddings) and step 5 (unembedding).

## 3 Attention
(ML notes pg 54-61)

#### 3.1 Query, Key, Value fundamentals
Every token embedding is projected into three different spaces:
- Query (Q): what this token is looking for
- Key (K): what this token offers
- Value (V): the information this token contributes if selected

All three come from the same input embedding x, but with different learned linear projections. The input token starts as the same vector x and is transformed using a learned weight matrix

**Mathematically:**
- Q = x Wq
- K = x Wk
- V = x Wv
  
Why this works:
- Attention becomes content-addressable memory.
- Tokens don’t attend by position or index, but by similarity in meaning.

Conceptual analogy:
- Query = question
- Key = label on a memory slot
- Value = content inside the slot


### 3.2. Dot product attention

The core operation:
```math
attention(Q, K, V) = softmax(Q Kᵀ / √dk) V
```

Step by step:
1. Compute similarity between every query and every key.
2. Normalize scores with softmax.
3. Use scores to form weighted sums of values.

This produces:
- For each token, a contextualized representation.
- Tokens can pull information from any previous token.

### 3.3 Why divide by √dk

Without scaling:
- Dot products grow with dimension.
- Softmax saturates (becomes near one-hot).
- Gradients vanish.

Scaling by **sqrt(dk)** keeps variance stable.

This is not optional. Training becomes unstable without it.


### 3.4 Causal masking and autoregressive behavior

LLMs generate left to right. A token must not see future tokens during training.

Causal mask:
- Upper triangular matrix filled with -inf
- Added to attention logits before softmax

Effect:
- Positions j > i get zero probability
- Guarantees autoregressive behavior

Conceptually:
- Training simulates generation
- Prediction at position i only depends on < i

**Similarity Scores Q Kᵀ (no mask)**
| Query \ Key | I    | love | ML   |
|------------|------|------|------|
| I          | 0.60 | 0.30 | 0.10 |
| love       | 0.20 | 0.50 | 0.30 |
| ML         | 0.10 | 0.67 | 0.23 |

**Casual mask**
| Query \ Key | I    | love | ML   |
|------------|------|------|------|
| I          | 1.00 | 0.00 | 0.00 |
| love       | 0.29 | 0.71 | 0.00 |
| ML         | 0.10 | 0.67 | 0.23 |

### 3.5 Softmax details and numerical stability

Softmax is:
$$
\text{softmax}(x_i) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}
$$

Numerical issue:
- Large logits overflow

Standard trick:
$$
\text{softmax}(x_i) = \frac{\exp(x_i - \max_j x_j)}{\sum_j \exp(x_j - \max_j x_j)}
$$
This preserves probabilities but stabilizes computation.

TensorFlow handles this internally, but you must remember why it works.

### 3.6 Multi-head attention
Single head attention has one similarity space basicaly it can only attend to one type of question. see `ML notes pg 61` for more

Multi-head attention:
- Splits channels into h heads
- Each head attends differently
- Results are concatenated and projected

Shape intuition:
```
(batch, seq, d_model)
→ (batch, heads, seq, d_head)
```
Where:
```
d_head = d_model / heads
```

Why this matters:
- Different heads learn syntax, semantics, coreference, long-range dependencies
- Heads act as independent subspaces

**Example**
Tokens: T1, T2, T3

Head 1 output:
| T1 | T2 | T3 |
|----|----|----|
| 0.1 | 0.3 | 0.6 |
| 0.2 | 0.5 | 0.3 |

Head 2 output:
| T1 | T2 | T3 |
|----|----|----|
| 0.4 | 0.4 | 0.2 |
| 0.3 | 0.3 | 0.4 |

Concatenated (along features):
| T1         | T2         | T3         |
|------------|------------|------------|
| 0.1, 0.4   | 0.3, 0.4   | 0.6, 0.2   |
| 0.2, 0.3   | 0.5, 0.3   | 0.3, 0.4   |

**Multi-Head Attention Combination**

Each head computes its own attention output independently:
$$
\text{head}_i = \text{Attention}(Q_i, K_i, V_i)
$$

Outputs of all heads are concatenated along the feature dimension:
$$
\text{Concat}(\text{head}_1, \ldots, \text{head}_h) \in \mathbb{R}^{d_{\text{model}}}
$$

A final learned linear projection mixes them:
$$
\text{MHA\_output} = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) \cdot W_O
$$

Note: attention scores are not combined across heads; each head attends separately.

### 3.7 Attention complexity

Time and memory:
```O(seq_len²)```

This is the main scaling bottleneck in LLMs.

Implications:
- Long context is expensive
- Motivates FlashAttention, sparse attention, sliding windows


### 3.8 Memory layout and tensor shapes (critical)

Most bugs in attention come from shape mistakes.

Typical shapes:
- Q, K, V: (batch, heads, seq_len, d_head) # d_head = d_model / heads
- Attention scores: (batch, heads, seq_len, seq_len)
- Output: (batch, seq_len, d_model)

Reshaping and transposing correctly is essential.


### 3.9 Flash Attention (conceptual only)

Flash Attention:
- Computes attention without materializing full seq_len × seq_len matrix
- Uses tiling and fused kernels
- Reduces memory bandwidth bottleneck

Important takeaway:
- Same math, different execution
- You do not change the model, only the kernel


### 3.10 Self attention vs cross attention

- Self attention: Q, K, V come from same sequence
- Cross attention: Q from decoder, K/V from encoder

GPT style LLMs use only self attention.

Encoder–decoder models use both.


### 3.11 Key-value caching for generation

During autoregressive generation:
- Past K and V do not change
- Only compute Q for new token
- Append new K, V to cache

Effect:
- Reduces per-token cost from O(n²) to O(n)


This is essential for fast inference.


### 3.12 Tensorflow Example (no raw kernal high level usage)

#### 3.12.1 Single-head attention (conceptual TF)

In [ ]:
import tensorflow as tf

def single_head_attention(x, mask=None):
    d_model = x.shape[-1]

    q = tf.keras.layers.Dense(d_model)(x)
    k = tf.keras.layers.Dense(d_model)(x)
    v = tf.keras.layers.Dense(d_model)(x)

    scores = tf.matmul(q, k, transpose_b=True)
    scores /= tf.math.sqrt(tf.cast(d_model, tf.float32))

    if mask is not None:
        scores += (mask * -1e9)

    weights = tf.nn.softmax(scores, axis=-1)
    output = tf.matmul(weights, v)
    return output

#### 3.12.2 Multi-head attention (high level)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model // num_heads,
            dropout=dropout
        )

    def call(self, x, mask=None, training=False):
        return self.mha(
            query=x,
            value=x,
            key=x,
            attention_mask=mask,
            training=training
        )


#### 3.12.3 Causal mask

In [ ]:
def causal_mask(seq_len):
    mask = tf.linalg.band_part(
        tf.ones((seq_len, seq_len)), -1, 0
    )
    return 1.0 - mask

#### 3.12.4 Multi-Head Attention in Action: Example 

In [ ]:
import tensorflow as tf
import keras_nlp
import numpy as np

# Load pretrained GPT-2
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
backbone = keras_nlp.models.GPT2Backbone.from_preset("gpt2_base_en")

# Interesting sentence with clear relationships
text = "The cat chased the mouse"

# Tokenize
token_ids = tokenizer(text)
tokens = [tokenizer.detokenize([tid]).numpy().decode() if isinstance(tokenizer.detokenize([tid]), tf.Tensor) 
          else tokenizer.detokenize([tid]) for tid in token_ids.numpy()]

if tf.rank(token_ids) == 1:
    token_ids = token_ids[None, :]

# Get embeddings
x_tokens = backbone.token_embedding(token_ids)
x_pos = backbone.position_embedding(x_tokens)
embeddings = x_tokens + x_pos

# Pass through FIRST transformer layer (uses pretrained attention weights!)
transformer_layer = backbone.transformer_layers[0]
output_embeddings = transformer_layer(embeddings)

# NEXT WORD PREDICTION (after just 1 layer)

# Unembed: project back to vocabulary
embedding_matrix = backbone.token_embedding.embeddings
logits = tf.matmul(output_embeddings, embedding_matrix, transpose_b=True)

# Get top 3 predictions for the LAST position (next word)
last_logits = logits[0, -1, :]
top3_ids = tf.argsort(last_logits, direction='DESCENDING')[:3]

#### 3.12.5 What Transformers Do and what do (context integration)
**The transformer's job when updating embeddings:**

Each transformer layer refines token embeddings by **integrating context from surrounding tokens** via attention:
- Before: Each token's embedding is isolated (just word + position info)
- After: Each token's embedding incorporates information from relevant context
- Example: "mouse" embedding gets updated based on "cat chased" → now encodes "something being chased"

**Why this matters for prediction:**

Updated embeddings → better next-token predictions. GPT-2 stacks 12 layers because:
- **1 layer**: Minimal context integration → weak predictions
- **12 layers**: Deep reasoning across all context → strong predictions

**Logit scores:** Higher number = more likely next token (passed through softmax to get probabilities)

## 4 Feed Forward Networks (MLP block)
ML notes pages 62-68

### 4.1 What the FFN is really doing

In a transformer layer, attention answers:

“Which tokens should talk to each other?”

The FFN answers:

“What computation should I apply to each token independently? its basically where your questions are answerd and facts are stored”

Key property:
- The FFN is position-wise.
- Same MLP is applied to every token.
- No interaction across sequence here, only across channels.

Mathematically, for each token vector x:
```math
FFN(x) = W2 · σ(W1 · x + b1) + b2
```
the W2 and B2 are for projecting the linear step W1 · x + b1 back down to a lower dim, the lenaer step is made up of a wight matrix and bias just like a NN and just liek a NN it also has a activation function σ to add non linearity

This is where:
- Nonlinearity enters the model.
- Feature composition happens.
- Model capacity explodes.

Most of the parameters in modern LLMs live here, not in attention.


### 4.2 Two-layer MLP structure

Standard configuration:
- Input dimension: d_model
- Hidden dimension: d_ff (usually 4× d_model)
- Output dimension: d_model

Example for GPT-like models:
- d_model = 768
- d_ff = 3072

Why expand then contract:
- Expansion creates a high-dimensional feature space.
- Nonlinearity selects and combines features.
- Projection back keeps residual dimensions stable.


### 4.3 Activation functions and why GELU matters
#### 4.3.1 ReLU
`ReLU(x) = max(0, x)`

Pros:
- Simple
- Fast

Cons:
- Hard cutoff at zero
- Less expressive for language

#### 4.3.2 GELU (Gaussian Error Linear Unit)
```
GELU(x) = x · Φ(x)
```
Where Φ is the standard normal CDF.

Intuition:
- Smooth gating
- Small negative values partially pass
- Better gradient flow

Why LLMs prefer GELU:
- Language is continuous, not sparse
- Smooth activations model nuance better
- Empirically improves perplexity

GPT-2, BERT, and many successors use GELU.


### 4.4 Modern variants: SwiGLU
This is where newer LLMs (PaLM, LLaMA, Mistral) improve capacity.

**Core idea**
Instead of one activation, split the hidden layer and gate it:
```
FFN(x) = (W1 x ⊙ σ(W2 x)) W3
```

For SwiGLU:
- σ is Swish: x · sigmoid(x)
- ⊙ means element-wise multiplication
- The gate controls information flow

Why this works:
- Explicit multiplicative interaction
- Better feature selection
- More expressive than plain GELU

Important note:
- Hidden dimension is often reduced (e.g. 2/3 of 4×) to keep parameter count similar.


### 4.5 Dropout and residual scaling

Dropout:
- Applied after activation or projection
- Regularizes training
- Less critical in massive pretraining, more in fine-tuning

Residual scaling:
- Output of FFN is added to residual stream
- Large activations can destabilize training
- Some models scale FFN output by a small constant early in training

Core pattern:
`x = x + dropout(FFN(LN(x)))`


### 4.6 TensorFlow: Configurable FFN block (high level)

In [ ]:
import tensorflow as tf

class FeedForward(tf.keras.layers.Layer):
    """ 
    Feed-Forward Network with configurable activation.
    Supports ReLU, GELU, and SwiGLU activations.
    """
    def __init__(
        self,
        d_model,
        d_ff,
        activation="gelu",
        dropout=0.1
    ):
        super().__init__()

        self.activation = activation
        self.dropout = tf.keras.layers.Dropout(dropout)

        if activation == "swiglu":
            self.w1 = tf.keras.layers.Dense(d_ff)
            self.w2 = tf.keras.layers.Dense(d_ff)
            self.w3 = tf.keras.layers.Dense(d_model)
        else:
            self.dense1 = tf.keras.layers.Dense(d_ff)
            self.dense2 = tf.keras.layers.Dense(d_model)

    def call(self, x, training=False):
        if self.activation == "relu":
            h = tf.nn.relu(self.dense1(x))
            h = self.dropout(h, training=training)
            return self.dense2(h)

        if self.activation == "gelu":
            h = tf.nn.gelu(self.dense1(x))
            h = self.dropout(h, training=training)
            return self.dense2(h)

        if self.activation == "swiglu":
            h = self.w1(x)
            gate = tf.nn.swish(self.w2(x))
            h = h * gate
            h = self.dropout(h, training=training)
            return self.w3(h)


using it inside the transformer layer (EX)
```py
ffn = FeedForward(
    d_model=768,
    d_ff=3072,
    activation="gelu"
)

y = ffn(x, training=True)
```

## 5. Normalization and Stabilization in Transformers

ML notes page 117

### 5.1 Why normalization is critical in deep transformers

Transformers stack dozens to hundreds of layers. Each layer:
- Applies attention
- Applies an FFN
- Adds residual connections (You add the layer’s input back to its output, this way Each layer learns a small correction, not a full transformation.)

```
DEFINITION:
Residual stream = the running hidden state that flows through the model via residual connections.
Short definition: It is the vector being repeatedly updated as: x ← x + sublayer_output at every layer.
```

Without normalization:
- Activations grow with depth
- Variance drifts layer by layer
- Gradients either explode or vanish
- Training becomes extremely sensitive to learning rate and initialization

Normalization exists to control the statistics of the residual stream so depth becomes feasible.

**The residual stream must remain well-conditioned across layers.**


### 5.2 LayerNorm inside transformers

**What LayerNorm actually does**

LayerNorm normalizes across features, not across batch.

For a token vector `x ∈ R^d_model`:

```
LN(x) = (x − mean(x)) / sqrt(var(x) + ε) * γ + β
```

Important properties:
- Applied per token
- Independent of batch size
- Ideal for sequence models

This is why transformers do not use BatchNorm.

**Where LayerNorm is applied**

In transformers, LayerNorm is applied inside each block, not globally.

**NOTE:** Two main placements exist. This choice matters a lot.


### 5.3 Post-Norm vs Pre-Norm

#### 5.3.1 Post-Norm (original Transformer)
Structure:
```
x → Attention → Add → LayerNorm
x → FFN → Add → LayerNorm
```
Add = residual connection, It means: take the input x and add it element-wise to the sublayer output, Sublayer output = the result of the operation inside the block before the residual add.

Problems:
- Gradients must pass through many nonlinear layers before normalization
- Deep models become unstable
- Training requires careful warmup and small learning rates

This works for shallow models but breaks at scale.

#### 5.3.2 Pre-Norm (modern standard)
Structure:
```
x → LayerNorm → Attention → Add
x → LayerNorm → FFN → Add
```

Why this works better:
- Residual path is clean and unnormalized
- Gradients flow directly through identity connections
- Much more stable for deep stacks

Key insight:
- Pre-norm turns the transformer into a well-behaved residual network.
- Almost all modern LLMs use Pre-Norm.


### 5.4 RMSNorm

**What RMSNorm is**

RMSNorm removes mean-centering and only normalizes by root mean square.

`RMSNorm(x) = x / rms(x) * γ`

Where:
`rms(x) = sqrt(mean(x²))`

Differences from LayerNorm:
- No subtraction of mean
- Fewer operations
- Slightly faster
- Numerically simpler

**Why modern LLMs prefer RMSNorm**

Empirical findings:
- Mean-centering is not strictly necessary
- RMS scaling alone stabilizes training
- Works well with large batch sizes and long contexts

Used in:
- LLaMA
- Mistral
- PaLM


RMSNorm is not about expressiveness. It is about efficiency and stability at scale.


### 5.5 Why normalization stabilizes attention and FFNs

Two failure modes normalization prevents:

1. Attention collapse
- QKᵀ grows too large
- Softmax saturates
- One token dominates
2. FFN blow-up
- Expansion layer produces large activations
- Residual accumulation amplifies them

Normalization ensures:
- Attention logits stay in a reasonable range
- FFN outputs do not dominate the residual stream
- Each layer operates in a similar statistical regime

This makes hyperparameters transferable across depths.


### 5.6 Initialization strategies for stability

Initialization is tightly coupled with normalization.

Key principles:
- Linear layers should preserve variance
- Residual paths should start near identity
- Early training should be conservative

Common strategies:
- Xavier or Kaiming for dense layers
- Small initialization for output projections
- Zero or near-zero initialization for some residual branches

In GPT-2:
- Output projection of attention and FFN initialized with smaller std
- This slows early residual accumulation

The goal:
- Let the model learn depth gradually.


### 5.7 Residual connection depth scaling

As depth increases, residual additions accumulate.

If each layer adds a signal of similar magnitude:
`||x_L|| ≈ L · ||Δ||`

This is bad.

Solutions:
- Scale residual outputs by a constant
- Implicit scaling via normalization
- Careful initialization

GPT-2 style scaling:
- Attention and FFN outputs are scaled implicitly via initialization
- Residual stream remains stable even at 48+ layers

Some modern models explicitly scale residuals by **1/√N.**

$$
x_{l+1} = x_l + \frac{1}{\sqrt{N}} \, \text{Sublayer}(x_l)
$$


### 5.8 TensorFlow: Pre-Norm transformer components (high level)

LayerNorm block
``` py
norm = tf.keras.layers.LayerNormalization(epsilon=1e-5)

y = norm(x)
```

In [ ]:
import tensorflow as tf

class RMSNorm(tf.keras.layers.Layer):
    def __init__(self, d_model, epsilon=1e-8):
        super().__init__()
        self.epsilon = epsilon
        self.scale = self.add_weight(
            shape=(d_model,),
            initializer="ones",
            trainable=True
        )

    def call(self, x):
        rms = tf.sqrt(tf.reduce_mean(tf.square(x), axis=-1, keepdims=True))
        return x / (rms + self.epsilon) * self.scale


Pre-Norm pattern inside a block
```py
x = x + attention(norm1(x))
x = x + ffn(norm2(x))
```

## 6 The Transformer Decoder Block

### 6.1 Decoder block order and why it is this way

A decoder block is a transformer layer that takes a sequence of token representations, applies causal self-attention and a feedforward network with residual connections and normalization, and outputs updated representations for next-token prediction.

Modern GPT style `decoder block` that uses Pre-Norm:
```
x → LayerNorm → Self-Attention → Dropout → Residual Add
  → LayerNorm → FFN → Dropout → Residual Add
```

Why this order matters:
- Normalization happens before heavy computation
- Residual path stays clean
- Gradients flow directly through depth
- Training remains stable even at 100+ layers

Attention always comes before FFN because:
- First mix information across tokens
- Then transform that information per token

### 6.2 Shape propagation through the block

Let:
- batch = B
- sequence length = T
- model dimension = D
- number of heads = H
- head dimension = D / H

Input:
`x: (B, T, D)`

After attention:
```
Q, K, V: (B, H, T, D/H)
attention output: (B, T, D)
```

After FFN:
```
expanded: (B, T, d_ff)
projected back: (B, T, D)
```

Critical rule: The residual stream shape never changes.


### 6.3 Dropout behavior inside the decoder

Dropout appears in three places conceptually:
1. Attention weights
2. Attention output projection
3. FFN output

Dropout is:
- Active only during training
- A regularizer, not a core mechanism

In large pretraining runs:
- Dropout is often small or zero
- In fine-tuning: Dropout becomes important again

**NOTE:** Always apply dropout before residual addition.


### 6.4 Trainable parameter count intuition
Per decoder block, roughly:

Attention:
- QKV projections: 3 × D × D
- Output projection: D × D

FFN:
- Expansion: D × d_ff
- Projection: d_ff × D

If **d_ff = 4D**:

Total per layer is dominated by FFN:
`~ 12D² parameters`

This is why:
- Most parameters live in FFNs
- Reducing FFN size dramatically reduces model size
- Attention is compute-heavy but parameter-light


### 6.5 How to tune hidden sizes

Rules of thumb:
- d_model: controls representation capacity
- num_heads: controls diversity of attention patterns
- d_ff: controls nonlinear capacity

Common ratios:
- d_ff = 4 × d_model for GELU
- d_ff ≈ 2.7 × d_model for SwiGLU

Increasing depth tends to be more effective than increasing width for language modeling.


### 6.6 Why decoder-only models work for LLMs

Decoder-only transformers:
- Predict next token given past tokens
- Train with causal self-attention
- Match generation exactly at training time

Key advantages:
- Simpler architecture
- No encoder-decoder mismatch
- Scales cleanly with data and compute

This alignment between training and inference is a major reason GPT style models dominate.


### 6.7 TensorFlow: Full DecoderBlock implementation

In [ ]:
import tensorflow as tf

class DecoderBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        d_model,
        num_heads,
        d_ff,
        dropout=0.1,
        activation="gelu"
    ):
        super().__init__()

        self.ln1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.ln2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)

        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model // num_heads,
            dropout=dropout
        )

        self.ffn = FeedForward(
            d_model=d_model,
            d_ff=d_ff,
            activation=activation,
            dropout=dropout
        )

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, causal_mask=None, training=False):
        # Self attention block
        h = self.ln1(x)
        h = self.attention(
            query=h,
            value=h,
            key=h,
            attention_mask=causal_mask,
            training=training
        )
        h = self.dropout(h, training=training)
        x = x + h

        # Feed forward block
        h = self.ln2(x)
        h = self.ffn(h, training=training)
        h = self.dropout(h, training=training)
        x = x + h

        return x

Testing with random input

**NOTE:** Run the FeedForward Code block in `4.6` before running the cell below

In [ ]:
batch = 2
seq_len = 8
d_model = 64

x = tf.random.normal((batch, seq_len, d_model))

causal_mask = tf.linalg.band_part(
    tf.ones((seq_len, seq_len)), -1, 0
)

block = DecoderBlock(
    d_model=d_model,
    num_heads=4,
    d_ff=256
)

y = block(x, causal_mask=causal_mask, training=True)
print(y.shape)


output:
```
(2, 8, 64)
```

**If this shape is preserved, the block is correct.**

## 7 Putting blocks together into a Transformer Decoder

### 7.1 Stacking N decoder blocks

A transformer decoder is simply:
- Token embedding
- Positional representation
- A stack of identical decoder blocks
- Final normalization
- Output projection to vocabulary

There is no recursion, no branching, no hierarchy. Just depth.

LLM Structure with `N Decoder Blocks`:
```
tokens
→ token embedding
→ positional embedding
→ [DecoderBlock × N] # xl+1 = DecoderBlock(xl​) for l=0…N−1 simply, the result from one decoder block is passed to the next one as input
→ final norm
→ linear projection (unembedding)
→ logits
```

Why stacking works:
- Each block refines the same residual stream
- Early layers learn local and syntactic features
- Middle layers learn structure and relations
- Late layers learn semantics and prediction cues

Depth increases abstraction, not sequence length.


### 7.2 Why weights are not shared across layers

You might ask why not reuse the same decoder block weights N times.

Reasons:
- Each layer needs to learn a different transformation
- Sharing collapses representational hierarchy
- Empirically hurts performance
- Transformers rely on progressive feature refinement. Weight sharing removes that ladder.

The only common sharing is:
- Token embedding ↔ output projection (tied embeddings)


### 7.3 How input embeddings flow into the decoder

Input tokens are integers. The decoder never sees integers again.

Flow:

1. Token IDs → embedding vectors
2. Embeddings scaled by sqrt(d_model)
3. Positional information added or applied
4. Result becomes the initial residual stream

From here on:
- Shape stays (B, T, D)
- All blocks operate on the same stream
- No block knows about tokens, only vectors

This is critical to internalize.


### 7.4 How logits are produced

After the final decoder block:
- We normalize once more
- Project each token vector to vocab size

Logits:

`(B, T, vocab_size)`

These logits represent:
- For every position
- A probability distribution over the next token

Softmax is applied later during loss computation or generation.


### 7.5 TensorFlow: Tiny GPT style model from scratch
This is a clean, minimal, but correct GPT style forward pass.

Assumptions:
- Pre-Norm decoder blocks
- Causal self attention
- Tied embeddings

In [ ]:
import tensorflow as tf

class TinyGPT(tf.keras.Model):
    def __init__(
        self,
        vocab_size,
        max_seq_len,
        d_model,
        num_heads,
        d_ff,
        num_layers, # of decoder blocks
        dropout=0.1
    ):
        super().__init__()

        self.token_embedding = tf.keras.layers.Embedding(
            vocab_size, d_model
        )

        self.position_embedding = tf.keras.layers.Embedding(
            max_seq_len, d_model
        )

        self.dropout = tf.keras.layers.Dropout(dropout)

        self.blocks = [
            DecoderBlock(
                d_model=d_model,
                num_heads=num_heads,
                d_ff=d_ff,
                dropout=dropout
            )
            for _ in range(num_layers)
        ]

        self.final_norm = tf.keras.layers.LayerNormalization(
            epsilon=1e-5
        )

    def call(self, token_ids, training=False):
        batch_size, seq_len = tf.shape(token_ids)[0], tf.shape(token_ids)[1]

        # Create causal mask
        causal_mask = tf.linalg.band_part(
            tf.ones((seq_len, seq_len)), -1, 0
        )

        # Embeddings
        x = self.token_embedding(token_ids)
        x *= tf.math.sqrt(tf.cast(x.shape[-1], tf.float32))

        positions = tf.range(seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb

        x = self.dropout(x, training=training)

        # Decoder blocks
        for block in self.blocks:
            x = block(
                x,
                causal_mask=causal_mask,
                training=training
            )

        # Final normalization
        x = self.final_norm(x)

        # Tied output projection
        logits = tf.einsum(
            "btd,vd->btv",
            x,
            self.token_embedding.embeddings
        )

        return logits

Testing the forward pass

In [ ]:
vocab_size = 1000
max_seq_len = 32

model = TinyGPT(
    vocab_size=vocab_size,
    max_seq_len=max_seq_len,
    d_model=64,
    num_heads=4,
    d_ff=256,
    num_layers=4
)

dummy_tokens = tf.random.uniform(
    (2, 16),
    minval=0,
    maxval=vocab_size,
    dtype=tf.int32
)

logits = model(dummy_tokens, training=True)
print(logits.shape)

Output:
`(2, 16, 1000)`

this confirms the logits (possible next words) are being generated hence the model is working corectly

## 8 Training Objective and Loss for LLMs

At a high level, LLM training is brutally simple:
- Given tokens up to position t, predict token t+1.

Everything else is engineering to make this stable and efficient.

see ml notes pg 69-70

### 8.1 Next-token prediction objective

The model defines:

`P(xₜ | x₁, x₂, …, xₜ₋₁)`

Training maximizes the log-likelihood of the dataset under this factorization.

Log-likelihood measures how well a model's predicted probability distribution matches the actual observed data, with higher values indicating better fit.

Equivalent view:
- The model outputs logits for every position.
- Each position predicts the next token.
- Loss is summed across all tokens in the batch.

This aligns training exactly with generation behavior. This alignment is a major reason decoder-only models dominate.


### 8.2 Cross-entropy loss in detail

For a single token prediction:

`loss = -log softmax(logits)[true_token]`

- `−logp(true token)` is the loss function itself
- `logits` are the raw, unnormalized scores that can be any real number.
- `softmax(logits)` gives a probibility distrabution over the logits where each probability is [0,1] and all together they add to 1
- [true_token] is to index the probability of the correct token from the probabilities given by softmax

Expanded:
- Softmax converts logits into probabilities.
- Cross-entropy penalizes low probability assigned to the true token.
- Loss is high when the model is confident and wrong.

Important properties:
- Loss is additive across time steps.
- Loss is additive across batch elements.
- Lower loss corresponds to better next-token prediction.

In practice, we never compute softmax explicitly. We use a numerically stable fused op.


### 8.3 Label shifting

The model outputs logits for each input token position.

But the target is the next token.

So we shift:

Input tokens:
- [BOS, A, B, C]

Targets:
- [A, B, C, EOS]

Implementation rule:
- Inputs are tokens[:-1]
- Targets are tokens[1:]

The model never sees the token it is trying to predict.


### 8.4 Padding removal in the loss

Sequences are padded to form batches.

Padding tokens must not contribute to the loss.

Mechanism:
- Create a mask where target != PAD
- Multiply loss by mask
- Normalize by number of real tokens

If you skip this:
- Model learns to predict PAD
- Loss statistics become meaningless
- Training degrades silently


### 8.5 Teacher forcing

Teacher forcing means:

During training, the model always receives the true previous token.

It never feeds its own predictions back during training.

Why this is correct:
- Objective is conditional likelihood
- This gives unbiased gradients
- Generation-time mismatch is handled by scale, not architecture

All GPT-style models use teacher forcing.


### 8.6 Gradient clipping and why it is essential (ML notes p 123)

Transformers can produce large gradients due to:
- Attention score spikes
- FFN expansion layers
- Long sequences

Gradient clipping prevents:
- Sudden divergence
- NaNs during early training
- Instability when scaling batch size

Standard practice:
- Clip by global norm
- Typical values: 0.5 to 1.0

This is not optional for LLMs.


### 8.7 Mixed precision training

Mixed precision uses:
- float16 or bfloat16 for most operations
- float32 for accumulations and loss

Benefits:
- Faster training
- Lower memory usage
- Larger batch sizes

Key requirement:
- Loss scaling to avoid underflow

TensorFlow handles this automatically when configured correctly.



#### 8.7.1 Loss Scaling

Loss scaling means multiplying the loss by a constant before backpropagation to keep gradients numerically stable.

Why it exists:
- In mixed precision training, gradients can underflow to zero in float16.
- Scaling the loss makes gradients larger so they stay representable.

What actually happens:
Compute loss
Multiply by scale factor **S**
Backpropagate
Divide gradients by **S**

It rescales the math so small gradients do not disappear.

### 8.8 TensorFlow: Custom autoregressive training loop

This is a clean, minimal training loop for a GPT-style model.

This loop:
- Implements next-token prediction
- Handles padding correctly
- Uses teacher forcing
- Clips gradients
- Trains with mixed precision

This is the same structure used in real LLM training codebases.


#### 8.8.1 Enable mixed precision

In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

# Using float32 for simplicity (mixed precision can cause issues with @tf.function)
mixed_precision.set_global_policy("float32")


#### 8.8.2 Loss function with padding mask

In [ ]:
def compute_loss(logits, targets, pad_token_id):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True,
        reduction="none"
    )

    loss = loss_fn(targets, logits)

    mask = tf.cast(targets != pad_token_id, tf.float32)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

#### 8.8.3 Custom training step


In [ ]:
@tf.function
def train_step(model, optimizer, tokens, pad_token_id):
    inputs = tokens[:, :-1]
    targets = tokens[:, 1:]

    with tf.GradientTape() as tape:
        logits = model(inputs, training=True)
        loss = compute_loss(logits, targets, pad_token_id)

    # Compute gradients
    grads = tape.gradient(loss, model.trainable_variables)

    # Gradient clipping
    grads, _ = tf.clip_by_global_norm(grads, 1.0)

    optimizer.apply_gradients(
        zip(grads, model.trainable_variables)
    )

    return loss

#### 8.8.4 Optimizer setup

In [ ]:
# Using simple AdamW optimizer
optimizer = tf.keras.optimizers.AdamW(
    learning_rate=3e-4,
    beta_1=0.9,
    beta_2=0.95,
    weight_decay=0.1
)


#### 8.8.5 Load built-in dataset and create a small model

In [ ]:
# Use IMDB reviews dataset (built-in with TensorFlow)
from tensorflow.keras.datasets import imdb

# Load top 5000 words only
vocab_size = 5000
max_length = 50

(train_texts, _), _ = imdb.load_data(num_words=vocab_size)

# Take small subset for demo (100 samples)
train_texts = train_texts[:100]

# Clip token IDs to be within vocab range (some tokens may be exactly vocab_size)
train_texts = [[min(token, vocab_size-1) for token in seq] for seq in train_texts]

# Pad sequences
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    train_texts, 
    maxlen=max_length,
    padding='post',
    value=0  # pad token
)

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices(padded_sequences).batch(4)

# Simple model for this vocab
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Dense(vocab_size)
])

#### 8.8.6 Training loop

In [ ]:
# Training loop
max_steps = 25

for step, batch_tokens in enumerate(dataset):
    if step >= max_steps:
        break
        
    loss = train_step(
        model,
        optimizer,
        batch_tokens,
        pad_token_id=0
    )

    if step % 5 == 0:
        tf.print("Step:", step, "Loss:", loss)

print(f"\nTraining complete! Ran {min(step+1, max_steps)} steps.")

Output:
```
Step: 0 Loss: 8.51775169
Step: 5 Loss: 8.51544094
Step: 10 Loss: 8.51357555
Step: 15 Loss: 8.51210499
Step: 20 Loss: 8.50906372

Training complete! Ran 25 steps.
```

## 9 Generation Controls for LLMs 

NOTE:
- Given a sequence of tokens, iteratively predict the next token and append it, until a stopping condition is met.
- Everything else (temperature, top-k, top-p, repetition penalty) is a mechanism for controlling diversity and coherence.

(ML notes p 94)

### 9.1 Greedy decoding

Concept:
- At each step, pick the token with the highest probability.
- `next_token = argmax(P(x_t | x_<t))`

Pros:
- Deterministic
- Fast
Cons:
- Can be repetitive
- Lacks diversity

Analogy: Always choose the most obvious next word.


### 9.2 Sampling
Instead of argmax, sample from the probability distribution:

`next_token ~ P(x_t | x_<t)`

Pros:
- Introduces variety
- Can generate creative sequences

Cons:
- Can produce low-quality or incoherent tokens if probability mass is diffuse


### 9.3 Temperature (ml notes p 53)

Temperature T scales logits before softmax:

`P(x) ∝ exp(logits / T)`

- T < 1 → sharper distribution → more deterministic
- T = 1 → unmodified
- T > 1 → flatter distribution → more randomness

Temperature controls risk vs safety in predictions.


### 9.4 Top-k sampling

Top-k limits sampling to the k most probable tokens:
1. Sort logits
2. Keep top k tokens
3. Renormalize probabilities
4. Sample

Effect:
- Prevents rare nonsense tokens from appearing
- Controls diversity explicitly


### 9.5 Top-p (nucleus) sampling

Top-p keeps the smallest set of tokens whose cumulative probability ≥ p:
- Example: p = 0.9
- Only sample from tokens covering 90% probability mass
- Dynamic per step, unlike fixed k

Advantage:
- Adaptive to uncertainty
- Avoids both overly deterministic or overly random generation


### 9.6 Repetition penalty

- Penalizes tokens that have appeared recently
- Multiplies logits of repeated tokens by a factor < 1
- Helps prevent loops like “the the the…”


### 9.7 Length penalty

- Penalizes sequences that are too short or too long
- Often applied during beam search (less common in standard autoregressive sampling)
- Encourages more natural-length output


### 9.8 Stopping conditions

- Maximum sequence length
- EOS token generation
- Custom stop strings
- Prevent infinite loops


### 9.9 Key-value cache for speed

During generation:
- Self-attention recomputes K/V for all previous tokens every step → quadratic cost
- Cache previous K/V tensors
- Only compute Q for new token
- Append new K/V to cache

Effect:
- Reduces per-step cost from O(T²) → O(T)
- Essential for real-time inference


### 9.10 Generation in tf.function

- Wrapping generation in tf.function converts Python loops into graph ops
- Greatly improves speed
- Important for long sequences


### 9.11 TensorFlow: Full autoregressive generation function

In [ ]:
import tensorflow as tf

def generate(
    model,
    input_ids,
    max_new_tokens,
    temperature=1.0,
    top_k=None,
    top_p=None,
    repetition_penalty=None,
    eos_token_id=None
):
    input_ids = tf.identity(input_ids)
    batch_size = tf.shape(input_ids)[0]
    cache = []

    for step in tf.range(max_new_tokens):
        logits = model(input_ids, training=False)  # shape (B, T, V)
        next_token_logits = logits[:, -1, :]

        # Repetition penalty
        if repetition_penalty is not None:
            for b in range(batch_size):
                for token in input_ids[b].numpy():
                    next_token_logits = tf.tensor_scatter_nd_update(
                        next_token_logits,
                        indices=[[b, token]],
                        updates=[next_token_logits[b, token] / repetition_penalty]
                    )

        # Temperature
        next_token_logits /= temperature

        # Top-k
        if top_k is not None:
            values, _ = tf.math.top_k(next_token_logits, k=top_k)
            min_values = tf.reduce_min(values, axis=-1, keepdims=True)
            next_token_logits = tf.where(
                next_token_logits < min_values, -1e10, next_token_logits
            )

        # Top-p
        if top_p is not None:
            sorted_logits = tf.sort(next_token_logits, direction='DESCENDING', axis=-1)
            sorted_probs = tf.nn.softmax(sorted_logits, axis=-1)
            cum_probs = tf.cumsum(sorted_probs, axis=-1)
            mask = cum_probs > top_p
            # Set masked logits to -inf
            sorted_indices = tf.argsort(next_token_logits, direction='DESCENDING')
            scatter_mask = tf.scatter_nd(
                tf.expand_dims(sorted_indices, -1),
                tf.cast(mask, tf.float32) * -1e10,
                tf.shape(next_token_logits)
            )
            next_token_logits += scatter_mask

        # Sample next token
        next_token = tf.random.categorical(next_token_logits, 1, dtype=tf.int32)
        input_ids = tf.concat([input_ids, next_token], axis=-1)

        # Stop if EOS token generated
        if eos_token_id is not None:
            if tf.reduce_all(tf.reduce_any(next_token == eos_token_id, axis=-1)):
                break

    return input_ids


## 10 Scaling and regularization

### 10.1 Parameter scaling laws

Empirical scaling laws show that loss follows a power law with respect to:
- Model parameters
- Dataset size
- Compute

Key insight:
- For a fixed compute budget, there is an optimal balance between model size and data size.

Practical consequences:
- Bigger models underfit if data is too small
- Smaller models saturate early if data is abundant
- Token count matters more than raw dataset size

Rule of thumb:
- Train on at least 20 to 30 tokens per parameter
- Under that, you are compute rich but data poor

This is why modern training focuses on trillions of tokens, not just billions of parameters.

### 10.2 Dropout placement

Dropout is not used everywhere in transformers.

Effective locations:
- Attention output projection
- FFN output projection
- Residual stream after sublayers

Usually avoided:
- Inside attention softmax
- Inside normalization layers

Typical values:
- 0.1 for small models
- 0.0 to 0.05 for very large models

Key insight:
- Dropout fights co-adaptation, but large datasets already provide strong regularization.


### 10.3 Weight decay

Weight decay penalizes large weights to prevent overfitting.

Important distinction:
- L2 regularization couples with optimizer
- Weight decay is applied directly to weights

Modern LLMs use AdamW, not Adam.

What to decay:
- Dense layer weights
- Projection matrices

What NOT to decay:
- Biases
- LayerNorm or RMSNorm parameters
- Embedding scale parameters

Typical values:
- 0.01 to 0.1


### 10.4 Learning rate schedules

Learning rate choice dominates training success.

Key idea:
- Large models are extremely sensitive to early training instability.

This is why schedules matter more than raw LR values.


### 10.5 Warmup linear schedules

Warmup increases learning rate gradually at the start.

Why it matters:
- Early gradients are noisy
- Large initial updates can destroy representations

Common strategy:
- Linear warmup for first 1 to 5 percent of total steps
- Then decay

Warmup is mandatory for transformers.


### 10.6 Cosine decay

After warmup, learning rate decays smoothly:

`lr(t) = lr_max * 0.5 * (1 + cos(πt / T))`

Benefits:
- Smooth convergence
- Avoids sudden drops
- Works well with long training runs

Often combined with:
- Warmup + cosine decay



### 10.7 Adam vs AdamW

Adam:
- Couples L2 regularization with adaptive moments
- Poor weight decay behavior

AdamW:
- Decouples weight decay from gradient updates
- Stable at scale
- Industry standard for LLMs

Typical hyperparameters:
- beta1 = 0.9
- beta2 = 0.95 or 0.999
0 epsilon = 1e-8


### 10.8 Tokenization impact on scale

Tokenization affects:
- Sequence length
- Effective context usage
- Compute cost

Observations:
- Byte-level tokenizers increase sequence length
- Subword tokenizers reduce length but increase vocab size
- Larger vocab increases embedding and softmax cost

Tradeoff:
- Smaller vocab → longer sequences
- Larger vocab → more parameters

This directly impacts:
- Memory
- Throughput
- Training stability


### 10.9 Transformer depth and expressivity

Depth increases expressivity more than width for a fixed parameter budget.

Why:
- Each layer refines the residual stream
- Attention compounds information across layers
- Deeper models build hierarchical abstractions

But depth increases:
- Gradient instability
- Optimization difficulty

This is why:
- Pre-norm
- Residual scaling
- Careful initialization are mandatory for deep LLMs.


### 10.10 Putting it together: real vs toy training

Toy models:
- Few layers
- Small datasets
- Minimal regularization
- Short training runs

Real LLMs:
- Careful LR schedules
- AdamW with selective decay
- Minimal dropout
- Massive datasets
- Gradient clipping
- Mixed precision
- Continuous evaluation

Core insight:
- Architecture gets you expressivity. Optimization keeps it alive.


## 11 Evaluation and Metrics for LLMs

### 11.1 Perplexity

Perplexity measures how surprised the model is by the data.

Definition:

`Perplexity = exp(cross_entropy_loss)`

where: exp = e^x

Interpretation:
- Lower perplexity means better predictive modeling
- It is the geometric mean of branching factor per token

Intuition:
- If your model has perplexity 20, it behaves like it is choosing among 20 plausible next tokens at each step.

Important nuances:
- Only meaningful when evaluated with the same tokenizer
- Sensitive to sequence length
- Cannot compare across tokenization schemes

Perplexity is the standard metric for language modeling quality.


### 11.2 Bits per character (BPC)

BPC is perplexity measured at the character level.

`BPC = cross_entropy_loss / log(2)`

Why it exists:
- Character tokenization varies widely
- BPC normalizes across tokenizers

Use cases:
- Comparing byte-level models
- Evaluating compression-like efficiency
- Lower BPC means better compression of language.


### 11.3 Loss curves

Loss curves show training dynamics, not final quality.

What to watch:
- Smooth downward trend
- No sharp spikes during warmup
- Validation loss tracks training loss

Red flags:
- Training loss decreases but validation loss rises → overfitting
- Loss plateaus early → under-capacity or learning rate too low
- Sudden spikes → optimizer instability

Loss curves are the fastest debugging tool.


### 11.4 Out-of-distribution evaluation

LLMs fail silently on distribution shifts.

OOD examples:
- Different domain text
- Different language style
- Longer context than trained on

Symptoms:
- Confident but wrong predictions
- Repetitive output
- Degenerate loops

Best practice:
- Evaluate on multiple domains
- Include longer contexts
- Monitor perplexity degradation

OOD evaluation is more important than in-distribution scores.


### 11.5 Token-level accuracy

Definition:
- Fraction of tokens where argmax prediction equals ground truth

Why it is misleading:
- Language is multimodal
- Multiple valid next tokens exist

Use it for:
- Debugging alignment
- Verifying pipeline correctness

Do not use it as a quality metric.

### 11.6 Sampling-based evaluation

Quantitative metrics miss qualitative failure modes.

Sampling evaluation includes:
- Prompted generation
- Long-context continuation
- Style consistency tests

What to check:
- Coherence over long spans
- Repetition behavior
- Sensitivity to temperature
- Robustness to small prompt changes

This is where most issues surface.


### 11.7 Practical evaluation workflow

A strong evaluation loop includes:
1. Perplexity on held-out data
2. Loss curve inspection
3. OOD perplexity tests
4. Sampled generations at multiple temperatures
5. Regression tests on known prompts

Key insight:
- No single metric correlates perfectly with usefulness.


### 11.8 When metrics lie

Situations where metrics mislead:
- Over-optimized perplexity harms creativity
- Tokenization differences skew comparisons
- Short context evaluation hides long-range failures

This is why modern LLM evaluation mixes:
- Automated metrics
- Human judgment
- Task-based benchmarks


### 11.9 End-to-end evaluation of a pretrained GPT-2 (TensorFlow)

In [ ]:
import tensorflow as tf
import keras_nlp
import numpy as np

# --------------------------------------------
# Load pretrained tokenizer and GPT-2 model
# --------------------------------------------
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset(
    "gpt2_base_en"
)

model = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en"
)

# --------------------------------------------
# Prepare evaluation text (in-distribution)
# --------------------------------------------
texts = [
    "Deep learning models rely on gradient based optimization.",
    "Transformers scale well with data and compute.",
    "The capital of France is Paris.",
    "Neural networks can fail silently on out of distribution data."
]

# --------------------------------------------
# Simpler approach: Evaluate each text independently
# --------------------------------------------
print("=== In-Distribution Evaluation ===")
losses = []
accuracies = []

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction="none"
)

for i, text in enumerate(texts):
    # Tokenize single text
    tokens = tokenizer(text)
    tokens_tensor = tf.convert_to_tensor([tokens], dtype=tf.int32)
    
    # Label shifting for next-token prediction
    input_ids = tokens_tensor[:, :-1]
    labels = tokens_tensor[:, 1:]
    
    # Create padding mask (all 1s since no padding in single sequence)
    padding_mask = tf.ones_like(input_ids)
    
    # Forward pass with token_ids and padding_mask
    logits = model({"token_ids": input_ids, "padding_mask": padding_mask}, training=False)
    
    # Compute loss
    loss_per_token = loss_fn(labels, logits)
    mean_loss = tf.reduce_mean(loss_per_token)
    perplexity = tf.exp(mean_loss)
    
    # Token accuracy (debugging metric) - cast to same dtype
    pred_tokens = tf.cast(tf.argmax(logits, axis=-1), tf.int32)
    token_accuracy = tf.reduce_mean(
        tf.cast(pred_tokens == labels, tf.float32)
    )
    
    losses.append(mean_loss.numpy())
    accuracies.append(token_accuracy.numpy())
    
    print(f"\nText {i+1}: {text[:50]}...")
    print(f"  Loss: {mean_loss.numpy():.4f}")
    print(f"  Perplexity: {perplexity.numpy():.4f}")
    print(f"  Token Accuracy: {token_accuracy.numpy():.4f}")

# Overall statistics
print(f"\n=== Overall In-Distribution Statistics ===")
print(f"Mean Loss: {np.mean(losses):.4f}")
print(f"Mean Perplexity: {np.exp(np.mean(losses)):.4f}")
print(f"Mean Token Accuracy: {np.mean(accuracies):.4f}")

# --------------------------------------------
# Sampling-based evaluation (generation)
# --------------------------------------------
print("\n=== Generation Evaluation ===")
print("(Skipped - generation may fail on some TensorFlow/Metal configurations)")
print("The model.generate() method may require additional backend configuration.")

# Alternative: Show next-token prediction for a prompt
prompt = "Large language models are"
print(f"\nPrompt: '{prompt}'")
tokens = tokenizer(prompt)
tokens_tensor = tf.convert_to_tensor([tokens], dtype=tf.int32)
padding_mask = tf.ones_like(tokens_tensor)

logits = model({"token_ids": tokens_tensor, "padding_mask": padding_mask}, training=False)
next_token_logits = logits[0, -1, :]  # Get last token's predictions

# Get top 5 predictions
top_k = 5
top_indices = tf.argsort(next_token_logits, direction='DESCENDING')[:top_k]
print(f"\nTop {top_k} next token predictions:")
for i, idx in enumerate(top_indices.numpy()):
    token = tokenizer.detokenize([idx])
    prob = tf.nn.softmax(next_token_logits)[idx].numpy()
    print(f"  {i+1}. Token ID {idx}: '{token}' (prob: {prob:.4f})")

# --------------------------------------------
# OOD evaluation (distribution shift)
# --------------------------------------------
print("\n=== Out-of-Distribution Evaluation ===")
ood_texts = [
    "The eigenvalues of a Hermitian operator are real.",
    "Quantum field theory describes particle interactions.",
    "In category theory, a functor maps objects to objects."
]

ood_losses = []
ood_perplexities = []

for i, text in enumerate(ood_texts):
    tokens = tokenizer(text)
    tokens_tensor = tf.convert_to_tensor([tokens], dtype=tf.int32)
    
    input_ids = tokens_tensor[:, :-1]
    labels = tokens_tensor[:, 1:]
    padding_mask = tf.ones_like(input_ids)
    
    logits = model({"token_ids": input_ids, "padding_mask": padding_mask}, training=False)
    loss_per_token = loss_fn(labels, logits)
    mean_loss = tf.reduce_mean(loss_per_token)
    perplexity = tf.exp(mean_loss)
    
    ood_losses.append(mean_loss.numpy())
    ood_perplexities.append(perplexity.numpy())
    
    print(f"\nOOD Text {i+1}: {text[:50]}...")
    print(f"  Loss: {mean_loss.numpy():.4f}")
    print(f"  Perplexity: {perplexity.numpy():.4f}")

print(f"\n=== Overall OOD Statistics ===")
print(f"Mean OOD Loss: {np.mean(ood_losses):.4f}")
print(f"Mean OOD Perplexity: {np.mean(ood_perplexities):.4f}")
print(f"\n=== Comparison ===")
print(f"In-dist Perplexity: {np.exp(np.mean(losses)):.4f}")
print(f"OOD Perplexity: {np.mean(ood_perplexities):.4f}")
print(f"Perplexity Ratio (OOD/In-dist): {np.mean(ood_perplexities) / np.exp(np.mean(losses)):.2f}")

# --------------------------------------------
# Key Observations:
# - OOD perplexity is higher, indicating distribution shift
# - Token accuracy is low (not a meaningful quality metric)
# - Perplexity quantifies model uncertainty
# - Lower perplexity = better fit to the data distribution
# --------------------------------------------

# --------------------------------------------
# Notes:
# - Perplexity is tokenizer-dependent
# - Token accuracy is not a quality metric
# - Higher OOD perplexity indicates distribution shift
# - Need to ensure consistent dtypes (int32) for comparisons
# --------------------------------------------


Output:

```
=== In-Distribution Evaluation ===

Text 1: Deep learning models rely on gradient based optimi...
  Loss: 5.5754
  Perplexity: 263.8670
  Token Accuracy: 0.1250

Text 2: Transformers scale well with data and compute....
  Loss: 5.4709
  Perplexity: 237.6756
  Token Accuracy: 0.1250

Text 3: The capital of France is Paris....
  Loss: 3.7983
  Perplexity: 44.6244
  Token Accuracy: 0.1667

Text 4: Neural networks can fail silently on out of distri...
  Loss: 5.9228
  Perplexity: 373.4550
  Token Accuracy: 0.0909

=== Overall In-Distribution Statistics ===
Mean Loss: 5.1919
Mean Perplexity: 179.8023
Mean Token Accuracy: 0.1269

=== Generation Evaluation ===
(Skipped - generation may fail on some TensorFlow/Metal configurations)
The model.generate() method may require additional backend configuration.

Prompt: 'Large language models are'

Top 5 next token predictions:
  1. Token ID 407: ' not' (prob: 0.0475)
  2. Token ID 973: ' used' (prob: 0.0433)
  3. Token ID 635: ' also' (prob: 0.0399)
  4. Token ID 1695: ' available' (prob: 0.0293)
  5. Token ID 1690: ' often' (prob: 0.0268)

=== Out-of-Distribution Evaluation ===

OOD Text 1: The eigenvalues of a Hermitian operator are real....
  Loss: 4.8738
  Perplexity: 130.8193

OOD Text 2: Quantum field theory describes particle interactio...
  Loss: 4.6373
  Perplexity: 103.2607

OOD Text 3: In category theory, a functor maps objects to obje...
  Loss: 4.7473
  Perplexity: 115.2712

=== Overall OOD Statistics ===
Mean OOD Loss: 4.7528
Mean OOD Perplexity: 116.4504

=== Comparison ===
In-dist Perplexity: 179.8023
OOD Perplexity: 116.4504
Perplexity Ratio (OOD/In-dist): 0.65
```

## 12. Final Integration: Building a Tiny LLM End-to-End

**Goal**: Synthesize all concepts from this handbook into one complete pipeline - from raw text to a trained transformer that generates text.

**This is conceptual** - not a full project, but shows how all pieces fit together. You can later attempt something like NanoGPT with full understanding.

**Pipeline Overview**:
1. Load dataset and tokenize
2. Create sequences for next-token prediction
3. Serialize to TFRecord for efficient training
4. Build tf.data pipeline with optimizations
5. Build a tiny transformer model
6. Train with custom loop (explicit gradient steps)
7. Generate text with the trained model
8. Evaluate perplexity

**Why This Matters**: After this section, you'll understand the full stack of LLM training - data prep, model architecture, training loop, and evaluation.

### 12.1 Setup: Imports and Configuration

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
from pathlib import Path

# Configuration for our tiny LLM
CONFIG = {
    'vocab_size': 5000,      # Small vocabulary for demo
    'seq_length': 64,        # Context window
    'd_model': 128,          # Embedding dimension
    'num_heads': 4,          # Attention heads
    'num_layers': 2,         # Transformer blocks
    'dff': 512,              # Feed-forward dimension
    'dropout': 0.1,
    'batch_size': 32,
    'epochs': 5,
    'learning_rate': 1e-4
}

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

### 12.2 Step 1: Load Dataset and Tokenize

We'll use a simple text corpus (or you can use a small dataset like tiny shakespeare).

**Concepts Applied**:
- Character or subword tokenization
- Building vocabulary
- Converting text to token IDs

In [ ]:
# Simulate loading a text dataset (in practice, load from file)
sample_text = """
Machine learning is a subset of artificial intelligence. It focuses on building systems 
that learn from data. Deep learning uses neural networks with many layers. Transformers 
revolutionized natural language processing. Attention mechanisms allow models to focus on 
relevant parts of the input. Large language models are trained on massive text corpora.
Tokenization breaks text into manageable units. Embeddings represent tokens as vectors.
Positional encodings add sequence information. Self-attention computes relationships between tokens.
""" * 100  # Repeat to get more data

# Simple word-level tokenization (for demo; use BPE/WordPiece in practice)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=CONFIG['vocab_size'], oov_token='<UNK>')
tokenizer.fit_on_texts([sample_text])

# Convert text to sequences
token_ids = tokenizer.texts_to_sequences([sample_text])[0]

print(f"Original text length: {len(sample_text)} characters")
print(f"Token sequence length: {len(token_ids)} tokens")
print(f"Vocabulary size: {len(tokenizer.word_index)} unique tokens")
print(f"Sample tokens: {token_ids[:20]}")

### 12.3 Step 2: Create Sequences for Next-Token Prediction

**Language Modeling Task**: Given tokens `[t1, t2, ..., tn]`, predict `t(n+1)`

We create overlapping windows:
- Input: `[t1, t2, ..., t64]` 
- Target: `[t2, t3, ..., t65]` (shifted by 1)

In [ ]:
def create_sequences(token_ids, seq_length):
    """
    Create input-target pairs for next-token prediction.
    
    Input:  [0, 1, 2, 3, 4, 5]  with seq_length=3
    Creates:
        Input: [[0, 1, 2], [1, 2, 3], [2, 3, 4]]
        Target: [[1, 2, 3], [2, 3, 4], [3, 4, 5]]
    """
    sequences = []
    for i in range(len(token_ids) - seq_length):
        input_seq = token_ids[i:i + seq_length]
        target_seq = token_ids[i + 1:i + seq_length + 1]
        sequences.append((input_seq, target_seq))
    
    return sequences

# Create training sequences
sequences = create_sequences(token_ids, CONFIG['seq_length'])

# Convert to numpy arrays
input_seqs = np.array([seq[0] for seq in sequences], dtype=np.int32)
target_seqs = np.array([seq[1] for seq in sequences], dtype=np.int32)

print(f"Created {len(sequences)} training examples")
print(f"Input shape: {input_seqs.shape}")
print(f"Target shape: {target_seqs.shape}")
print(f"\nExample:")
print(f"  Input:  {input_seqs[0][:10]}")
print(f"  Target: {target_seqs[0][:10]}")

### 12.4 Step 3: Create TFRecord for Efficient Training

**Why TFRecord?**
- Efficient storage and loading
- Better performance with large datasets
- Integrates seamlessly with tf.data

In [ ]:
def serialize_example(input_seq, target_seq):
    """Convert numpy arrays to TFRecord format."""
    feature = {
        'input': tf.train.Feature(int64_list=tf.train.Int64List(value=input_seq)),
        'target': tf.train.Feature(int64_list=tf.train.Int64List(value=target_seq))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# Write to TFRecord
tfrecord_path = '/tmp/tiny_llm_data.tfrecord'
with tf.io.TFRecordWriter(tfrecord_path) as writer:
    for input_seq, target_seq in zip(input_seqs, target_seqs):
        serialized = serialize_example(input_seq, target_seq)
        writer.write(serialized)

print(f"Wrote {len(sequences)} examples to {tfrecord_path}")
print(f"File size: {os.path.getsize(tfrecord_path) / 1024:.2f} KB")

### 12.5 Step 4: Build tf.data Pipeline

**Optimizations Applied**:
- Parse TFRecord
- Shuffle for better training
- Batch efficiently
- Prefetch to overlap data loading with training
- Cache for repeated epochs

In [ ]:
def parse_tfrecord(serialized_example):
    """Parse a single TFRecord example."""
    feature_description = {
        'input': tf.io.FixedLenFeature([CONFIG['seq_length']], tf.int64),
        'target': tf.io.FixedLenFeature([CONFIG['seq_length']], tf.int64)
    }
    parsed = tf.io.parse_single_example(serialized_example, feature_description)
    return tf.cast(parsed['input'], tf.int32), tf.cast(parsed['target'], tf.int32)

# Build optimized pipeline
def create_dataset(tfrecord_path, batch_size, shuffle_buffer=10000):
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(tfrecord_path, CONFIG['batch_size'])

# Verify dataset
for inputs, targets in train_dataset.take(1):
    print(f"Batch input shape: {inputs.shape}")
    print(f"Batch target shape: {targets.shape}")
    print(f"Input sample: {inputs[0, :10]}")
    print(f"Target sample: {targets[0, :10]}")

### 12.6 Step 5: Build Tiny Transformer Model

**Architecture**:
- Token + Positional Embeddings
- 2 Transformer Decoder Blocks (Multi-Head Attention + FFN)
- Causal masking (can't attend to future tokens)
- Output layer: project to vocabulary logits

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    """Sinusoidal positional encoding."""
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
        
        pe = np.zeros((max_len, d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        
        self.pe = tf.constant(pe[np.newaxis, :, :], dtype=tf.float32)
    
    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pe[:, :seq_len, :]

class TransformerBlock(tf.keras.layers.Layer):
    """Single transformer decoder block with causal attention."""
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=d_model // num_heads
        )
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization()
        self.layernorm2 = tf.keras.layers.LayerNormalization()
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, training=False):
        # Causal mask: position i can only attend to positions <= i
        seq_len = tf.shape(x)[1]
        causal_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        
        # Self-attention with causal mask
        attn_output = self.attention(x, x, attention_mask=causal_mask, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        # Feed-forward
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TinyLLM(tf.keras.Model):
    """Tiny transformer language model."""
    def __init__(self, vocab_size, d_model, num_heads, num_layers, dff, dropout=0.1):
        super().__init__()
        self.token_embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model)
        self.dropout = tf.keras.layers.Dropout(dropout)
        
        self.transformer_blocks = [
            TransformerBlock(d_model, num_heads, dff, dropout)
            for _ in range(num_layers)
        ]
        
        self.output_layer = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x, training=False):
        # x: (batch_size, seq_len)
        x = self.token_embedding(x)  # (batch_size, seq_len, d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)
        
        for block in self.transformer_blocks:
            x = block(x, training=training)
        
        logits = self.output_layer(x)  # (batch_size, seq_len, vocab_size)
        return logits

# Build model
model = TinyLLM(
    vocab_size=CONFIG['vocab_size'],
    d_model=CONFIG['d_model'],
    num_heads=CONFIG['num_heads'],
    num_layers=CONFIG['num_layers'],
    dff=CONFIG['dff'],
    dropout=CONFIG['dropout']
)

# Print model summary
model.build(input_shape=(None, CONFIG['seq_length']))
print(model.summary())
print(f"\nTotal parameters: {model.count_params():,}")

### 12.7 Step 6: Train with Custom Loop

**Why Custom Training Loop?**
- Full control over gradient computation
- Easier debugging
- Custom metrics and logging
- Understanding what happens under the hood

**Process**:
1. Forward pass → compute logits
2. Calculate loss (sparse categorical cross-entropy)
3. Backward pass → compute gradients
4. Update weights with optimizer

In [ ]:
# Setup for training
optimizer = tf.keras.optimizers.Adam(CONFIG['learning_rate'])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

@tf.function
def train_step(inputs, targets):
    """Single training step with gradient tape."""
    with tf.GradientTape() as tape:
        # Forward pass
        logits = model(inputs, training=True)
        
        # Calculate loss (averaged over sequence and batch)
        loss = loss_fn(targets, logits)
    
    # Backward pass
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Update metrics
    train_loss(loss)
    train_accuracy(targets, logits)
    
    return loss

# Training loop
print("Starting training...")
for epoch in range(CONFIG['epochs']):
    start_time = time.time()
    
    # Reset metrics (use reset_state for newer Keras versions)
    train_loss.reset_state()
    train_accuracy.reset_state()
    
    # Train on batches
    for step, (inputs, targets) in enumerate(train_dataset):
        loss = train_step(inputs, targets)
        
        if step % 10 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss:.4f}")
    
    # Epoch summary
    epoch_time = time.time() - start_time
    print(f"\nEpoch {epoch+1}/{CONFIG['epochs']} completed in {epoch_time:.2f}s")
    print(f"  Loss: {train_loss.result():.4f}")
    print(f"  Accuracy: {train_accuracy.result():.4f}")
    print("-" * 50)

print("Training completed!")

### 12.8 Step 7: Generate Text

**Autoregressive Generation**:
1. Start with a prompt (seed tokens)
2. Model predicts next token
3. Append predicted token to sequence
4. Repeat

**Sampling Strategies**:
- Greedy: Always pick highest probability token
- Top-k: Sample from k most likely tokens
- Temperature: Control randomness (high temp = more random)

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=50, temperature=1.0, top_k=10):
    """
    Generate text autoregressively.
    
    Args:
        prompt: Starting text
        max_length: Number of tokens to generate
        temperature: Controls randomness (lower = more deterministic)
        top_k: Sample from top k tokens
    """
    # Tokenize prompt
    token_ids = tokenizer.texts_to_sequences([prompt])[0]
    
    # Ensure we have at least some tokens
    if len(token_ids) == 0:
        token_ids = [1]  # Start with some token
    
    generated = token_ids.copy()
    
    for _ in range(max_length):
        # Take last seq_length tokens as context
        context = generated[-CONFIG['seq_length']:]
        context = tf.constant([context])
        
        # Pad if needed
        if len(context[0]) < CONFIG['seq_length']:
            padding = tf.zeros((1, CONFIG['seq_length'] - len(context[0])), dtype=tf.int32)
            context = tf.concat([padding, context], axis=1)
        
        # Predict next token
        logits = model(context, training=False)
        logits = logits[0, -1, :] / temperature  # Last position logits with temperature
        
        # Top-k sampling
        top_k_logits, top_k_indices = tf.nn.top_k(logits, k=top_k)
        top_k_probs = tf.nn.softmax(top_k_logits)
        
        # Sample from top-k
        next_token_idx = tf.random.categorical([top_k_probs], num_samples=1)[0, 0]
        next_token = top_k_indices[next_token_idx].numpy()
        
        generated.append(int(next_token))
    
    # Decode back to text
    # Note: tokenizer.sequences_to_texts expects list of sequences
    generated_text = tokenizer.sequences_to_texts([generated])[0]
    return generated_text

# Test generation
prompts = [
    "machine learning",
    "deep learning uses",
    "attention mechanisms"
]

print("=" * 60)
print("TEXT GENERATION EXAMPLES")
print("=" * 60)

for prompt in prompts:
    print(f"\nPrompt: '{prompt}'")
    print("-" * 60)
    
    # Generate with different temperatures
    for temp in [0.5, 1.0, 1.5]:
        generated = generate_text(model, tokenizer, prompt, max_length=30, temperature=temp, top_k=10)
        print(f"[Temperature={temp}]: {generated}")
    print()

Output Example:

```
Prompt: 'machine learning'
------------------------------------------------------------
[Temperature=0.5]: machine learning is on computes self building text models tokens positional encodings networks information many layers into tokens attention layers transformers allow focus tokens attention learn encodings the large into data models
[Temperature=1.0]: machine learning is self learning subset information focuses are add positional tokenization positional tokenization breaks intelligence large into of positional breaks systems that learn from tokens layers to natural into language processing
```

### 12.9 Step 8: Evaluate Perplexity

**Perplexity**: Measures how well the model predicts the test data
- Lower perplexity = better model
- Formula: `perplexity = exp(average_loss)`
- Intuition: "How surprised is the model by the actual next token?"

**Why It Matters**:
- Standard metric for language models
- Comparable across different models
- Reflects model uncertainty

In [ ]:
def calculate_perplexity(model, dataset):
    """
    Calculate perplexity on a dataset.
    
    Perplexity = exp(average cross-entropy loss)
    """
    total_loss = 0.0
    num_batches = 0
    
    for inputs, targets in dataset:
        logits = model(inputs, training=False)
        loss = loss_fn(targets, logits)
        total_loss += loss.numpy()
        num_batches += 1
    
    avg_loss = total_loss / num_batches
    perplexity = np.exp(avg_loss)
    
    return perplexity, avg_loss

# Calculate perplexity on training data (in practice, use separate validation set)
perplexity, avg_loss = calculate_perplexity(model, train_dataset)

print("=" * 60)
print("MODEL EVALUATION")
print("=" * 60)
print(f"Average Loss: {avg_loss:.4f}")
print(f"Perplexity: {perplexity:.4f}")
print()
print("Interpretation:")
print(f"  - The model is approximately {perplexity:.2f}-ways 'confused' on average")
print(f"  - Lower perplexity indicates better predictions")
print(f"  - For reference:")
print(f"    * Random guessing with vocab_size={CONFIG['vocab_size']} → perplexity ≈ {CONFIG['vocab_size']}")
print(f"    * Perfect prediction → perplexity = 1.0")
print(f"    * GPT-3 on validation set → perplexity ≈ 20-30")

### 12.10 What We've Accomplished

✅ **Complete LLM Pipeline**:
1. **Data Prep**: Tokenized text → sequences → TFRecord
2. **Data Pipeline**: Efficient tf.data with batching, shuffling, prefetching
3. **Model**: Transformer with embeddings, positional encoding, multi-head attention
4. **Training**: Custom loop with explicit gradient computation
5. **Inference**: Autoregressive text generation with sampling strategies
6. **Evaluation**: Perplexity metric for model quality

**Key Concepts Mastered**:
- End-to-end flow from raw data to trained model
- How tokenization affects model vocabulary
- Sequence creation for causal language modeling
- Efficient data loading with TFRecord + tf.data
- Transformer architecture components working together
- Custom training loop mechanics (forward, backward, optimize)
- Generation strategies (temperature, top-k sampling)
- Model evaluation with perplexity

### **Next Steps to NanoGPT Level**

Now that you understand the full pipeline conceptually, you can:

1. **Scale Up**: 
   - Use larger datasets (e.g., OpenWebText, Wikipedia)
   - Increase model size (more layers, larger d_model)
   - Train longer with more compute

2. **Better Tokenization**:
   - Implement BPE (Byte-Pair Encoding)
   - Use SentencePiece or tiktoken
   - Optimize vocabulary size

3. **Advanced Training**:
   - Learning rate scheduling (warmup + decay)
   - Gradient clipping for stability
   - Mixed precision training (FP16)
   - Gradient accumulation for larger batches

4. **Model Improvements**:
   - Add more layers and parameters
   - Experiment with different attention patterns
   - Implement techniques like Flash Attention

5. **Evaluation**:
   - Multiple benchmarks (LAMBADA, HellaSwag, etc.)
   - Human evaluation
   - Downstream task fine-tuning

**You're now equipped to understand and build projects like NanoGPT!** 🚀

The key insight: It's all about moving tensors through transformations:
```
Text → Tokens → Embeddings → Attention → FFN → Logits → Predictions → Text
```

Every component serves a purpose in this chain, and you now understand how they fit together.